In [1]:
import math
import pandas as pd

pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

Create a pool class that can replay transactions

- Initalize
- Mint
- Burn
- Collect
- Swap

functions to read current state
loop through df of events and get state update with df for LP positions ect




In [2]:
#setup to pull from ccdata
#give will temp key to sort the LP address issue

In [3]:
pn = '0x4e68ccd3e89f51c3074ca5072bbac773960dfa36'

In [4]:
df = pd.read_parquet(f"/Volumes/luke-uni/data/uniswap-v3/raw/{pn}/{pn}_12000000_13000000.parquet")
df

,event,logIndex,transactionIndex,transactionHash,address,blockHash,blockNumber,args.sender,args.recipient,args.amount0,args.amount1,args.sqrtPriceX96,args.liquidity,args.tick,args.owner,args.tickLower,args.tickUpper,args.amount,args.observationCardinalityNextOld,args.observationCardinalityNextNew,args.paid0,args.paid1
0,Mint,31,11,0x2e07c690f149223e4f290986277304ea6a05c6ee47ba303732166bc1b15cbafb,0x4e68Ccd3E89f51C3074ca5072bbAC773960dFa36,0x7ba9aaae07e50875f15d2ef395d3ffe858bfb7516f408a61af303deeb83e18f1,12375326,0xC36442b4a4522E871399CD717aBDD847Ab11FE88,None,1.242324e+19,5.050028e+10,NaN,NaN,NaN,0xC36442b4a4522E871399CD717aBDD847Ab11FE88,-197400.0,-193320.0,8.210905e+15,NaN,NaN,NaN,NaN
1,Initialize,27,11,0x2e07c690f149223e4f290986277304ea6a05c6ee47ba303732166bc1b15cbafb,0x4e68Ccd3E89f51C3074ca5072bbAC773960dFa36,0x7ba9aaae07e50875f15d2ef395d3ffe858bfb7516f408a61af303deeb83e18f1,12375326,None,None,NaN,NaN,4.585618e+24,NaN,-195153.0,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Mint,307,212,0x85a3f0ed4fc14085609e95e7366364566b89d7569e62e4a828d1ef1e5dc81e58,0x4e68Ccd3E89f51C3074ca5072bbAC773960dFa36,0xf88bd25d065be98356225e94bbc932874168c8e0d4cd1b4e86932d5147b9c795,12375529,0xC36442b4a4522E871399CD717aBDD847Ab11FE88,None,1.000000e+15,0.000000e+00,NaN,NaN,NaN,0xC36442b4a4522E871399CD717aBDD847Ab11FE88,-195120.0,-195060.0,1.935471e+13,NaN,NaN,NaN,NaN
3,Mint,148,104,0x0fde2929c1fa5dd5dc9a0fcfbf8adccdeefbc398aa36ff019aa2b15eded23552,0x4e68Ccd3E89f51C3074ca5072bbAC773960dFa36,0x72c9b1ac035b3ba5b4aafa034d2236069397c62c810d49bb8b4d3a0db09d7c1f,12375553,0xC36442b4a4522E871399CD717aBDD847Ab11FE88,None,9.995372e+10,2.740000e+02,NaN,NaN,NaN,0xC36442b4a4522E871399CD717aBDD847Ab11FE88,-195180.0,-195120.0,3.509242e+09,NaN,NaN,NaN,NaN
4,Mint,105,45,0x9ece74425a9b5b937b51e14d4fbb9eb7aa6f6ae415d51a460b6f56335d31b9f7,0x4e68Ccd3E89f51C3074ca5072bbAC773960dFa36,0xf58014d8c69be2d526202b3435808161fbd15d6e3002d1ec184077e936e1ca06,12375566,0xC36442b4a4522E871399CD717aBDD847Ab11FE88,None,9.999750e+12,2.741200e+04,NaN,NaN,NaN,0xC36442b4a4522E871399CD717aBDD847Ab11FE88,-195180.0,-195120.0,3.510779e+11,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
116131,Swap,17,5,0x4bc014eda3414a0acdc460472b4922d36137cc28d4f95d6f18246331867c22ec,0x4e68Ccd3E89f51C3074ca5072bbAC773960dFa36,0x31e3784a0a2215213d91d31d8b38654fef6fae5f7408bac4fd739100d71cec8f,12999867,0xE592427A0AEce92De3Edee1F18E0157C05861564,0x56178a0d5F301bAf6CF3e1Cd53d9863437345Bf9,-1.309952e+20,4.169304e+11,4.464409e+24,1.206692e+19,-195689.0,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN
116132,Swap,58,52,0xe8fb7604f70bc1929af37e5490f1acb16bdd94a55aa2049a94b9fdcc94243e84,0x4e68Ccd3E89f51C3074ca5072bbAC773960dFa36,0x62ef6549899d9139334c1a9dbc89e612af2a1d9345453d162070e86899f585d0,12999868,0xE592427A0AEce92De3Edee1F18E0157C05861564,0x56178a0d5F301bAf6CF3e1Cd53d9863437345Bf9,-1.453121e+20,4.630952e+11,4.467441e+24,1.206692e+19,-195676.0,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN
116133,Swap,14,23,0xab4ce993489886deb90277344c5c6eb222b2f778a45eb779a35efc76b84294bc,0x4e68Ccd3E89f51C3074ca5072bbAC773960dFa36,0x8ab66e948204774ed84f548cb4669e25b20084acc32eb3eb8e91a8647e0f3036,12999882,0xE592427A0AEce92De3Edee1F18E0157C05861564,0x56178a0d5F301bAf6CF3e1Cd53d9863437345Bf9,1.144828e+20,-3.627121e+11,4.465059e+24,1.206692e+19,-195686.0,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN
116134,Swap,33,14,0x2d6ca9132fe2f1e0bf1fa600af6ad0d37348d155dd54e22dce1a5969a6b2a9df,0x4e68Ccd3E89f51C3074ca5072bbAC773960dFa36,0x1875edf8ab897a4da7e210ef89b5e9f509d579f71c48bd3fde1c3068eb7733dd,12999887,0xE592427A0AEce92De3Edee1F18E0157C05861564,0x56178a0d5F301bAf6CF3e1Cd53d9863437345Bf9,2.460449e+20,-7.782293e+11,4.459950e+24,1.206692e+19,-195709.0,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [26]:
df.sort_values(by = ['blockNumber', 'logIndex'], inplace=True)

In [27]:
#error classes
class TickPriceAllignmentError(Exception):
    """Raise when tick and sqrtPrice do not allign"""

class IncorrectInput(Exception):
    """Raise when input is incorrect"""

class SwapAllignmentError(Exception):
    """Raise when swap data is misalligned"""

class BurnMintMatchError(Exception):
    """Raise when a burn event cant be matched with a previous mint"""

class CollectMatchError(Exception):
    """Raise when a Collect event cant be matched with a active position"""

class SwapAmountError(Exception):
    """Raise when swap has two positive amounts"""

class FeeMismatch(Exception):
    """Raise when swap fee collected and distributed do not match"""



In [28]:

#TODO Fix fringe cases for matching the burns and collects
#TODO Get view/quote for swap amountin return end price trade price and amount
#TODO Blog on uniswap math functions with figures
#TODO Add protocol fee 

In [114]:
class ConcentratedLiquidity():
    """
    ConcentratedLiquidity implementation in Python to replay transactions and track LP profit.
    This class does not take into consideration the variable prescision as in the solidity implimentations
    """

    def __init__(self,
        token0,
        token1,
        poolAddress,
        fee,
        tickSpacing,
        protocol_fee = 0,
        ):
        """
        Parameters
        ----------

        token0  :   str
            Token address name for token0 in the pool
        token1  :   str
            Token address name for token0 in the pool
        poolAddress :   str
            Token address for the pool
        fee :   int
            Fee as in logs Initalise
        protocol_fee    :   float
            percentage of fee that goes to the protocol
        tickSpacing :   int
            tickspacing in the pool
        """
        
        self.token0 = token0
        self.token1 = token1
        self.poolAddress = poolAddress
        self.fee = fee*10**-6 #fee adjustment to get to a percentage fee
        self.tickSpacing = tickSpacing
        self.protocol_fee = protocol_fee*10**-6 #fee adjustment to get to a percentage fee
        self.positions = pd.DataFrame()
        self.mints = pd.DataFrame()
        self.burns = pd.DataFrame()
        self.collects = pd.DataFrame()
        self.swaps = pd.DataFrame()
        self.total_fee0 = 0
        self.total_fee1 = 0
        self.liquidity = 0
        self.Q96 = 2**96
        self.sqrtPrice = None 
        self.sqrtPriceX96 = None

    
    def Initialize(self, 
                   sqrtPrice = None, 
                   sqrtPriceX96 = None, 
                   price = None, 
                   tick = None,
                   ):
        """
        Parameters
        ----------

        sqrtPrice  :   float
            Initialized sqrtPrice for the pool this input takes precident over sqrtPriceX96 and price
        sqrtPriceX96 :   str
            Initialized sqrtPrice for the pool this input takes precident over price
        price :   int
            Initialized price for the pool
        tick  :   int
            Initialized tick for the pool
        """
                

        if not any([sqrtPrice, sqrtPriceX96, price]):
            raise IncorrectInput("Initialize: need to add a sqrtPrice, price or sqrtPriceX96")

        if price is not None:
            self.sqrtPrice = price**0.5
            self.sqrtPriceX96 = sqrtPrice*self.Q96
        elif sqrtPriceX96 is not None:
            self.sqrtPriceX96 = sqrtPriceX96
            self.sqrtPrice = float(self.sqrtPriceX96_to_sqrtPrice(sqrtPriceX96))
        elif sqrtPrice is not None:
            self.sqrtPrice = float(sqrtPrice)
            self.sqrtPriceX96 = sqrtPrice*self.Q96
        
        #check if tick is supplied, otherwise use the self calc. remove keep data, can not supply tick if dont want check
        if tick is not None:
            if self.sqrtPrice_to_tick(self.sqrtPrice) != tick:
                raise TickPriceAllignmentError(f"Initialize: The price and tick supplied do not match\n\n\t{tick}, {self.sqrtPrice_to_tick(self.sqrtPrice)}")
            else:
                self.tick = tick
        else:
            self.tick = self.sqrtPrice_to_tick(self.sqrtPrice)


    def Mint(self, tickLower, tickUpper, amount, amount0, amount1, sender, blockNumber, transactionIndex, logIndex, transactionHash):
        """
        Parameters
        ----------
        #TODO params
        sqrtPrice  :   float
            Initialized sqrtPrice for the pool this input takes precident over sqrtPriceX96 and price
        """

        
        mint_df = pd.DataFrame([['Mint', logIndex, blockNumber, transactionIndex, transactionHash, sender, amount, tickLower, tickUpper, amount0, amount1]], 
                       columns=['event', 'logIndex', 'blockNumber', 'transactionIndex', 'transactionHash', 'sender', 'amount', 'tickLower', 'tickUpper', 'amount0', 'amount1'])
        
        self.mints = pd.concat([self.mints, mint_df])

        #TODO Add calc of portfolio amounts from current L and price, check if amounts are correct
        #Precision errors with bit math leave alot to be desired        

        add_active_df = pd.DataFrame([[float(amount), float(amount), tickLower, tickUpper, sender,
                               float(amount0), float(amount1), float(amount0), float(amount1),
                               float(0),float(0),float(0),float(0),
                               logIndex, blockNumber, transactionIndex, transactionHash,
                               logIndex, blockNumber, transactionIndex, transactionHash, True]],
                               columns = ['last_L', 'start_L', 'tickLower', 'tickUpper', 'owner', 'start_token0_holdings', 'start_token1_holdings',
                                            'last_token0_holdings', 'last_token1_holdings',
                                            'token0_fees_accrued', 'token1_fees_accrued',
                                            'token0_collected', 'token1_collected',
                                            'start_logIndex', 'start_blockNumber', 'start_transactionIndex', 'start_transactionHash', 
                                            'last_logIndex', 'last_blockNumber', 'last_transactionIndex', 'last_transactionHash', 'active'])

        
        self.positions = pd.concat([self.positions, add_active_df]).reset_index(drop=True)

        #save intick liquidity
        pos = self.positions
        current_tick = self.sqrtPrice_to_tick(self.sqrtPrice)
        self.liquidity = pos['last_L'].loc[(pos['tickLower'] <= current_tick)&(pos['tickUpper'] > current_tick)].sum()

        
    def Burn(self, tickLower, tickUpper , amount, amount0, amount1, owner, blockNumber, transactionIndex, logIndex, transactionHash):
        """
        Parameters
        ----------
        #TODO params
        sqrtPrice  :   float
            Initialized sqrtPrice for the pool this input takes precident over sqrtPriceX96 and price
        """

        burn_df = pd.DataFrame([['Burn', logIndex, blockNumber, transactionIndex, transactionHash, owner, amount, tickLower, tickUpper, amount0, amount1]], 
                       columns=['event', 'logIndex', 'blockNumber', 'transactionIndex', 'transactionHash', 'sender', 'amount', 'tickLower', 'tickUpper', 'amount0', 'amount1'])
        
        self.burns = pd.concat([self.burns, burn_df])
        
        pos = self.positions
        #check if len >1, otherwise burn liquidity in pool
        bpos = pos.loc[(pos['tickLower'] == tickLower) & (pos['tickUpper'] == tickUpper)]
        if len(bpos) > 1:
            #TODO handle cases when bounds do not match
            blpos = bpos.loc[bpos['last_L'] == amount]
            if len(blpos) == 1:
                i = blpos.index[0]

            elif len(blpos) == 0:
                raise BurnMintMatchError("Cannot match mints and burns because there is no L with same tick range")
            
            else:
                raise BurnMintMatchError("Cannot match mints and burns more the one position with same L")
            
        elif len(bpos) == 0:
                raise BurnMintMatchError("Cannot match mints and burns because there is no L with same tick range")

        else:
            i = bpos.index[0]
        
        L_delta = bpos['last_L'].at[i]

        pos['last_L'].iat[i] -= L_delta #Burn only removes liquidity, if not collected the amount wont change 
        pos['last_token0_holdings'].iat[i]  = amount0
        pos['last_token1_holdings'].iat[i]  = amount1

    
        pos = self.position_last_update_state(pos, blockNumber, transactionIndex, logIndex, transactionHash)
        self.positions = pos.copy()
        return 
    
    
    def Collect(self, tickLower, tickUpper, amount0, amount1, recipient, blockNumber, transactionIndex, logIndex, transactionHash):
        """
        Parameters
        ----------
        #TODO params
        sqrtPrice  :   float
            Initialized sqrtPrice for the pool this input takes precident over sqrtPriceX96 and price
        """

        collect_df = pd.DataFrame([['Collect', logIndex, blockNumber, transactionIndex, transactionHash, recipient, tickLower, tickUpper, amount0, amount1]], 
                       columns=['event', 'logIndex', 'blockNumber', 'transactionIndex', 'transactionHash', 'sender', 'tickLower', 'tickUpper', 'amount0', 'amount1'])
        self.collects = pd.concat([self.collects, collect_df])

        pos = self.positions
        cpos = pos.loc[(pos['tickLower'] == tickLower) & (pos['tickUpper'] == tickUpper)]

        if len(cpos) > 1:
            #TODO handle cases when bounds do not match
            clpos = cpos.loc[(cpos['last_L'] == 0)&((cpos['last_token0_holdings'] > 0)|(cpos['last_token1_holdings'] > 0))] 
            if len(clpos) == 1:
                i = clpos.index[0]

            elif len(clpos) == 0:
                raise CollectMatchError("Cannot match mints and burns because there is no null L in same tick range")

            else:
                raise CollectMatchError("Cannot match mints and burns more the one position with same L")
        else:
            i = cpos.index[0]

        pos['token0_collected'].iat[i] += amount0
        pos['token1_collected'].iat[i] += amount1

        token0_diff = (pos['last_token0_holdings'].iloc[i] + pos['token0_fees_accrued'].iloc[i]  - pos['token0_collected'].iloc[i])
        token1_diff = (pos['last_token1_holdings'].iloc[i] + pos['token1_fees_accrued'].iloc[i]  - pos['token1_collected'].iloc[i])
        if (token0_diff <= 0) & (token1_diff <= 0): #Should really be ==
            pos['active'].iat[i] = False
        
        pos[['last_token0_holdings', 'last_token1_holdings']] = pos.apply(lambda x: self.get_amounts(self.sqrtPrice, self.tick_to_sqrtPrice(x.tickLower), self.tick_to_sqrtPrice(x.tickUpper), x.last_L), axis = 1, result_type='expand')
        pos = self.position_last_update_state(pos, blockNumber, transactionIndex, logIndex, transactionHash)
        self.positions = pos.copy()
        
        return
    
    def Swap(self, amount0, amount1,  sender, recipient, logIndex, blockNumber, transactionIndex, transactionHash, tick = None, liquidity = None):
        """
        Parameters
        ----------
        #TODO params
        sqrtPrice  :   float
            Initialized sqrtPrice for the pool this input takes precident over sqrtPriceX96 and price
        """

        swap_df = pd.DataFrame([['Swap', logIndex, blockNumber, transactionIndex, transactionHash, sender, recipient, amount0, amount1, tick, liquidity]], 
                       columns=['event', 'logIndex', 'blockNumber', 'transactionIndex', 'transactionHash', 'sender', 'recipient', 'amount0', 'amount1', 'tick', 'liquidity'])
        
        self.swaps = pd.concat([self.swaps, swap_df])

        #Fees for pool 
        zeroForOne = None
        if amount0 > 0:
            total_fee0 = amount0*self.fee
            amount0_nf = amount0 - total_fee0
            zeroForOne = True
        else:
            total_fee0 = 0
            amount0_nf = amount0

        if amount1 > 0:
            total_fee1= amount1*self.fee
            amount1_nf = amount1 - total_fee1
            zeroForOne = False
        else:
            total_fee1 = 0
            amount1_nf = amount1

        if zeroForOne == None:
            raise SwapAmountError(f"Swap amounts are incorrect:\n\n\t{(amount0,amount1)}")

        self.total_fee0 += total_fee0
        self.total_fee1 += total_fee1

        pos = self.positions
        current_tick, current_tick_lower, current_tick_upper = self.get_tick_range(self.tick)

        if zeroForOne:
            sqrtPrice = self.sqrtPrice 
            sqrtPriceA = self.tick_to_sqrtPrice(current_tick_lower) #based on lower tick not lower price

            amount0_a = amount0_nf
            amount1_a = amount1_nf

            fee0_collected = 0
            while amount0_a > 0:
                active_pos = pos.loc[(pos['tickLower'] < current_tick)&(pos['tickUpper'] >= current_tick)]
                L = int(active_pos['last_L'].astype(float).sum())

                #check if there is enough reserves in the tick
                if self.get_amount0(sqrtPrice, sqrtPriceA, L) > amount0_a:
                    sqrtPrice_next = self.get_next_sqrtPrice_from_inputs(sqrtPrice, L, amount0_a, zeroForOne=zeroForOne)
                    tick_next = self.sqrtPrice_to_tick(sqrtPrice_next)

                    fee0_in_range = round((amount0_a/(1-self.fee)) - amount0_a)
                    fee0_collected += fee0_in_range
                    fee0_per_L = fee0_in_range/L

                    pos['token0_fees_accrued'] = pos['token0_fees_accrued'].mask(pos.index.isin(active_pos.index), pos['token0_fees_accrued'] + pos['last_L'] * fee0_per_L)
                    break


                else:
                    amount0_diff = self.get_amount0(sqrtPrice, sqrtPriceA, L) 
                    amount1_diff = self.get_amount1(sqrtPrice, sqrtPriceA, L) 

                    fee0_in_range = round((amount0_diff/(1-self.fee)) - amount0_diff)
                    fee0_collected += fee0_in_range
                    fee0_per_L = fee0_in_range/L
                    pos['token0_fees_accrued'] = pos['token0_fees_accrued'].mask(pos.index.isin(active_pos.index), pos['token0_fees_accrued'] + pos['last_L'] * fee0_per_L)

                    amount0_a -= amount0_diff
                    amount1_a -= amount1_diff
                    current_tick = current_tick_lower
                    current_tick_lower = current_tick - self.tickSpacing
                    sqrtPrice = self.tick_to_sqrtPrice(current_tick)
                    sqrtPriceA = self.tick_to_sqrtPrice(current_tick_lower)

        else:
            sqrtPrice = self.sqrtPrice
            sqrtPriceB = self.tick_to_sqrtPrice(current_tick_upper) #based on upper tick not upper price

            amount0_a = amount0_nf
            amount1_a = amount1_nf

            fee1_collected = 0
            while amount1_a > 0:
                active_pos = pos.loc[(pos['tickLower'] <= current_tick)&(pos['tickUpper'] > current_tick)]
                L = int(active_pos['last_L'].astype(float).sum())

                #check if there is enough reserves in the tick
                if self.get_amount1(sqrtPrice, sqrtPriceB, L) > amount1_a:
                    sqrtPrice_next = self.get_next_sqrtPrice_from_inputs(sqrtPrice, L, amount1_a, zeroForOne=zeroForOne)
                    tick_next = self.sqrtPrice_to_tick(sqrtPrice_next)

                    fee1_in_range = round((amount1_a/(1-self.fee)) - amount1_a)
                    fee1_collected += fee1_in_range
                    fee1_per_L = fee1_in_range/L

                    pos['token1_fees_accrued'] = pos['token1_fees_accrued'].mask(pos.index.isin(active_pos.index), pos['token1_fees_accrued'] + pos['last_L'] * fee1_per_L)
                    break

                else:
                    amount0_diff = self.get_amount0(sqrtPrice, sqrtPriceB, L) 
                    amount1_diff = self.get_amount1(sqrtPrice, sqrtPriceB, L) 

                    fee1_in_range = round((amount1_diff/(1-self.fee)) - amount1_diff)
                    fee1_collected += fee1_in_range
                    fee1_per_L = fee1_in_range/L
                    pos['token1_fees_accrued'] = pos['token1_fees_accrued'].mask(pos.index.isin(active_pos.index), pos['token1_fees_accrued'] + pos['last_L'] * fee1_per_L)

                    amount0_a -= amount0_diff
                    amount1_a -= amount1_diff

                    current_tick = current_tick_upper
                    current_tick_upper = current_tick + self.tickSpacing
                    sqrtPrice = self.tick_to_sqrtPrice(current_tick)
                    sqrtPriceB = self.tick_to_sqrtPrice(current_tick_upper)


        if not any([liquidity, tick]): #save if check not given
            self.sqrtPrice = sqrtPrice_next
            self.tick = tick_next
            self.liquidity = L

        else:
            if (int(L) != liquidity) or (int(tick) != int(tick_next)):
                raise SwapAllignmentError(f"Swap: tick or liquidity provided does not match calculations\n\n\tliquidity: {int(L)}, {liquidity}\n\ttick: {int(tick)}, {int(tick_next)}")
            else:
                self.sqrtPrice = sqrtPrice_next
                self.tick = tick_next
                self.liquidity = L    

        #update positions for estimate portfolio holdings
        #It is an estimate due to precision errors
        pos[['last_token0_holdings', 'last_token1_holdings']] = pos.apply(lambda x: self.get_amounts(self.sqrtPrice, self.tick_to_sqrtPrice(x.tickLower), self.tick_to_sqrtPrice(x.tickUpper), x.last_L), axis = 1, result_type='expand')
        pos = self.position_last_update_state(pos, blockNumber, transactionIndex, logIndex, transactionHash)
        self.positions = pos.copy()
        return
    
    def get_active_LP_positions(self):
        positions = self.positions
        return positions.loc[(positions['active'])&(positions['last_L'] > 0)]
    
    def view_all_pool_events(self):
        pool_events = pd.concat([self.mints, self.burns, self.collects, self.swaps])
        if pool_events.empty:
            return  pool_events
        else:
            return pool_events.sort_values(['blockNumber', 'logIndex']).reset_index(drop=True)

    def sqrtPriceX96_to_sqrtPrice(self, sqrtPriceX96):
        return float(sqrtPriceX96 / self.Q96)

    def sqrtPrice_to_sqrtPriceX96(self, sqrtPrice):
        return float(sqrtPrice * self.Q96)
    
    def sqrtPrice_to_tick(self, sqrtPrice):
        return math.floor(round(math.log(sqrtPrice, math.sqrt(1.0001)), 6)) #control for precision issues and tick int size with the rounding
    
    def tick_to_sqrtPrice(self, tick):
        return float(1.0001 ** (tick / 2))
    
    def price(self):
        return self.sqrtPrice**2

    def get_tick_range(self, tick):
        return tick, (tick//self.tickSpacing * self.tickSpacing), tick//self.tickSpacing * self.tickSpacing + self.tickSpacing
    
    def position_last_update_state(self, position, blockNumber, transactionIndex, logIndex, transactionHash):
        position['last_blockNumber'] = blockNumber
        position['last_transactionIndex'] = transactionIndex
        position['last_logIndex'] = logIndex
        position['last_transactionHash'] = transactionHash
        return position
    
    def get_amount0(self, sqrtPriceA, sqrtPriceB, L):
        if sqrtPriceA > sqrtPriceB:
            sqrtPriceA, sqrtPriceB = sqrtPriceB, sqrtPriceA
        
        return L * ((1/sqrtPriceA) - (1/sqrtPriceB))

    def get_amount1(self, sqrtPriceA, sqrtPriceB, L):
        if sqrtPriceA > sqrtPriceB:
            sqrtPriceA, sqrtPriceB = sqrtPriceB, sqrtPriceA
        
        return (L * (sqrtPriceB - sqrtPriceA))

    def get_amounts(self, sqrtPrice, sqrtPriceA, sqrtPriceB, L):
        amount0 = 0
        amount1 = 0

        if sqrtPriceA > sqrtPriceB:
            sqrtPriceA, sqrtPriceB = sqrtPriceB, sqrtPriceA
        
        if sqrtPrice <= sqrtPriceA:
            amount0 = self.get_amount0(sqrtPriceA, sqrtPriceB, L)
        
        elif sqrtPrice < sqrtPriceB:
            amount0 = self.get_amount0(sqrtPrice, sqrtPriceB, L)
            amount1 = self.get_amount1(sqrtPriceA, sqrtPrice, L)
        
        else:
            amount1 = self.get_amount1(sqrtPriceA, sqrtPriceB, L)

        return amount0, amount1

    def get_next_sqrtPrice_from_amount0(self, sqrtPrice, L, amonutIn): 
        return 1/((amonutIn/L)+(1/sqrtPrice))
    
    def get_next_sqrtPrice_from_amount1(self, sqrtPrice, L, amonutIn): 
        return sqrtPrice + (amonutIn / L)
    
    def get_next_sqrtPrice_from_inputs(self, sqrtPrice, L, amonutIn, zeroForOne):
        if zeroForOne:
            sqrtPriceNext = self.get_next_sqrtPrice_from_amount0(sqrtPrice, L, amonutIn)
        else:
            sqrtPriceNext = self.get_next_sqrtPrice_from_amount1(sqrtPrice, L, amonutIn)
        return sqrtPriceNext

    def calc_L_from_amount0(self, amount, sqrtPriceA, sqrtPriceB):
        if sqrtPriceA > sqrtPriceB:
            sqrtPriceA, sqrtPriceB = sqrtPriceB, sqrtPriceA
        return amount/((1/sqrtPriceB)-(1/sqrtPriceA))

    def calc_L_from_amount1(self, amount, sqrtPriceA, sqrtPriceB):
        if sqrtPriceB > sqrtPriceA:
            sqrtPriceB, sqrtPriceA = sqrtPriceA, sqrtPriceB
        return amount / (sqrtPriceA - sqrtPriceB)

    def calc_L_from_amounts(self, amount0, amount1, sqrtPrice, tickLower, tickUpper): #only estimates due to precision errors
        sqrtPriceA = self.tick_to_sqrtPrice(tickLower)
        sqrtPriceB = self.tick_to_sqrtPrice(tickUpper)

        liquidity0 = self.calc_L_from_amount0(amount0, sqrtPrice, sqrtPriceB)
        liquidity1 = self.calc_L_from_amount1(amount1, sqrtPrice, sqrtPriceA)

        L = min(liquidity0, liquidity1)

        return int(L)
    
    def replay_from_logs_for_LP_profit(self, df):
        #TODO add frequency need timestamp
        tdf = df.loc[df['event'] == 'Initialize']
        self.Initialize(sqrtPriceX96 = tdf['args.sqrtPriceX96'].iloc[0], 
                        tick = tdf['args.tick'].iloc[0])

        pos_dfs = []
        for i in range(len(df)):
            tdf = df.iloc[i]

            if tdf['event'] == 'Swap':
                self.Swap(blockNumber = tdf['blockNumber'],
                        transactionIndex = tdf['transactionIndex'],
                        logIndex = tdf['logIndex'],
                        transactionHash = tdf['transactionHash'],
                        sender = tdf['args.sender'],
                        recipient = tdf['args.recipient'],
                        amount0 = tdf['args.amount0'],
                        amount1 = tdf['args.amount1'],
                        tick = tdf['args.tick'],
                        liquidity = tdf['args.liquidity'])


            if tdf['event'] == 'Collect':
                self.Collect(tickLower = tdf['args.tickLower'], 
                            tickUpper = tdf['args.tickUpper'], 
                            amount0 = tdf['args.amount0'],
                            amount1 = tdf['args.amount1'],
                            recipient = tdf['args.recipient'],
                            blockNumber = tdf['blockNumber'], 
                            transactionIndex = tdf['transactionIndex'], 
                            logIndex = tdf['logIndex'], 
                            transactionHash = tdf['transactionHash'])
                

            if tdf['event'] == 'Burn':
                self.Burn(tickLower = tdf['args.tickLower'], 
                        tickUpper = tdf['args.tickUpper'], 
                        amount = tdf['args.amount'],
                        amount0 = tdf['args.amount0'],
                        amount1 = tdf['args.amount1'],
                        owner = tdf['args.owner'],
                        blockNumber = tdf['blockNumber'], 
                        transactionIndex = tdf['transactionIndex'], 
                        logIndex = tdf['logIndex'], 
                        transactionHash = tdf['transactionHash'])

            if tdf['event'] == 'Mint':
                self.Mint(tickLower = tdf['args.tickLower'], 
                        tickUpper = tdf['args.tickUpper'], 
                        amount = tdf['args.amount'],
                        amount0 = tdf['args.amount0'],
                        amount1 = tdf['args.amount1'],
                        sender = tdf['args.sender'],
                        blockNumber = tdf['blockNumber'], 
                        transactionIndex = tdf['transactionIndex'], 
                        logIndex = tdf['logIndex'], 
                        transactionHash = tdf['transactionHash'])
            
            pos_dfs.append(self.positions)
        
        position_df = pd.concat(pos_dfs)
        position_df.drop_duplicates(subset=['last_L', 'start_L', 'tickLower', 'tickUpper', 'owner',
       'start_token0_holdings', 'start_token1_holdings',
       'last_token0_holdings', 'last_token1_holdings', 'token0_fees_accrued',
       'token1_fees_accrued', 'token0_collected', 'token1_collected', 'start_logIndex', 'start_blockNumber', 'start_transactionIndex',
       'start_transactionHash',], keep = 'first', inplace = True)
        return position_df
    
    def replay_from_CCData_for_LP_profit(self, ts_from, ts_to, CCAPI):
        #TODO add functionality to get the data from CCdata and return the positions df
        position_df = pd.DataFrame()
        return position_df

    

    
    

In [115]:
FEE = 3000
TICK_SPACING = 60
POOL = '0x4e68ccd3e89f51c3074ca5072bbac773960dfa36'
TOKEN0 = 'WETHU'
TOKEN1 = 'USDT'

clm = ConcentratedLiquidity(TOKEN0, TOKEN1, POOL, fee = FEE, tickSpacing = TICK_SPACING, protocol_fee = 0)

In [111]:
rdf = clm.replay_from_logs_for_LP_profit(df)
rdf

SwapAllignmentError: Swap: tick or liquidity provided does not match calculations

	liquidity: 41448871078369024, 4.144887107836903e+16
	tick: -194899, -194899

In [107]:
rdf['start_transactionHash'].unique() # can use the start txnhash to get the unique mints or the index

NameError: name 'rdf' is not defined

In [116]:
tdf = df.loc[df['event'] == 'Initialize']
clm.Initialize(sqrtPriceX96 = tdf['args.sqrtPriceX96'].iloc[0], 
                        tick = tdf['args.tick'].iloc[0])

pos_dfs = []
for i in range(len(df)):
    tdf = df.iloc[i]

    if tdf['event'] == 'Swap':
        clm.Swap(blockNumber = tdf['blockNumber'],
                 transactionIndex = tdf['transactionIndex'],
                logIndex = tdf['logIndex'],
                transactionHash = tdf['transactionHash'],
                sender = tdf['args.sender'],
                recipient = tdf['args.recipient'],
                amount0 = tdf['args.amount0'],
                amount1 = tdf['args.amount1'],
                tick = None,#tdf['args.tick'],
                liquidity = None)#tdf['args.liquidity'])

    if tdf['event'] == 'Collect':
        clm.Collect(tickLower = tdf['args.tickLower'], 
                    tickUpper = tdf['args.tickUpper'], 
                    amount0 = tdf['args.amount0'],
                    amount1 = tdf['args.amount1'],
                    recipient = tdf['args.recipient'],
                    blockNumber = tdf['blockNumber'], 
                    transactionIndex = tdf['transactionIndex'], 
                    logIndex = tdf['logIndex'], 
                    transactionHash = tdf['transactionHash'])

    if tdf['event'] == 'Burn':
        clm.Burn(tickLower = tdf['args.tickLower'], 
                tickUpper = tdf['args.tickUpper'], 
                amount = tdf['args.amount'],
                amount0 = tdf['args.amount0'],
                amount1 = tdf['args.amount1'],
                owner = tdf['args.owner'],
                blockNumber = tdf['blockNumber'], 
                transactionIndex = tdf['transactionIndex'], 
                logIndex = tdf['logIndex'], 
                transactionHash = tdf['transactionHash'])

    if tdf['event'] == 'Mint':
        clm.Mint(tickLower = tdf['args.tickLower'], 
                 tickUpper = tdf['args.tickUpper'], 
                 amount = tdf['args.amount'],
                 amount0 = tdf['args.amount0'],
                 amount1 = tdf['args.amount1'],
                 sender = tdf['args.sender'],
                 blockNumber = tdf['blockNumber'], 
                 transactionIndex = tdf['transactionIndex'], 
                 logIndex = tdf['logIndex'], 
                 transactionHash = tdf['transactionHash'])
    
    pos_dfs.append(clm.positions)
        
position_df = pd.concat(pos_dfs)
position_df.drop_duplicates(subset=['last_L', 'start_L', 'tickLower', 'tickUpper', 'owner', 'start_token0_holdings', 'start_token1_holdings', 'last_token0_holdings', 'last_token1_holdings', 'token0_fees_accrued', 'token1_fees_accrued', 'token0_collected', 'token1_collected', 'start_logIndex', 'start_blockNumber', 'start_transactionIndex', 'start_transactionHash'], 
                            keep = 'first', inplace = True)

        



BurnMintMatchError: Cannot match mints and burns because there is no L with same tick range

In [117]:
pos = clm.positions
pos

,last_L,start_L,tickLower,tickUpper,owner,start_token0_holdings,start_token1_holdings,last_token0_holdings,last_token1_holdings,token0_fees_accrued,token1_fees_accrued,token0_collected,token1_collected,start_logIndex,start_blockNumber,start_transactionIndex,start_transactionHash,last_logIndex,last_blockNumber,last_transactionIndex,last_transactionHash,active
0,8.210905e+15,8.210905e+15,-197400.0,-193320.0,0xC36442b4a4522E871399CD717aBDD847Ab11FE88,1.242324e+19,5.050028e+10,1.043564e+19,5.725322e+10,3.776977e+16,1.512614e+08,0.0,0.0,31,12375326,11,0x2e07c690f149223e4f290986277304ea6a05c6ee47ba303732166bc1b15cbafb,48,12378991,21,0x6f622a5ff42de9e903318e8aabf039625e1d0db55cf5ac2e522ca9bb86157757,True
1,1.935471e+13,1.935471e+13,-195120.0,-195060.0,0xC36442b4a4522E871399CD717aBDD847Ab11FE88,1.000000e+15,0.000000e+00,0.000000e+00,3.371107e+06,0.000000e+00,1.014375e+04,0.0,0.0,307,12375529,212,0x85a3f0ed4fc14085609e95e7366364566b89d7569e62e4a828d1ef1e5dc81e58,48,12378991,21,0x6f622a5ff42de9e903318e8aabf039625e1d0db55cf5ac2e522ca9bb86157757,True
2,3.509242e+09,3.509242e+09,-195180.0,-195120.0,0xC36442b4a4522E871399CD717aBDD847Ab11FE88,9.995372e+10,2.740000e+02,0.000000e+00,6.093916e+02,4.935618e+07,1.174541e+00,0.0,0.0,148,12375553,104,0x0fde2929c1fa5dd5dc9a0fcfbf8adccdeefbc398aa36ff019aa2b15eded23552,48,12378991,21,0x6f622a5ff42de9e903318e8aabf039625e1d0db55cf5ac2e522ca9bb86157757,True
3,3.510779e+11,3.510779e+11,-195180.0,-195120.0,0xC36442b4a4522E871399CD717aBDD847Ab11FE88,9.999750e+12,2.741200e+04,0.000000e+00,6.096585e+04,4.937780e+09,1.175055e+02,0.0,0.0,99,12375566,44,0x34ca11f7f1c6a4d8d17cabf14c6d0ffa5116a693508cd974c882e06308479820,48,12378991,21,0x6f622a5ff42de9e903318e8aabf039625e1d0db55cf5ac2e522ca9bb86157757,True
4,3.510779e+11,3.510779e+11,-195180.0,-195120.0,0xC36442b4a4522E871399CD717aBDD847Ab11FE88,9.999750e+12,2.741200e+04,0.000000e+00,6.096585e+04,4.937780e+09,1.175055e+02,0.0,0.0,105,12375566,45,0x9ece74425a9b5b937b51e14d4fbb9eb7aa6f6ae415d51a460b6f56335d31b9f7,48,12378991,21,0x6f622a5ff42de9e903318e8aabf039625e1d0db55cf5ac2e522ca9bb86157757,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
129,2.589468e+16,2.589468e+16,-196260.0,-193380.0,0xC36442b4a4522E871399CD717aBDD847Ab11FE88,3.168309e+19,1.019990e+11,3.168439e+19,1.019945e+11,2.235539e+14,7.565801e+05,0.0,0.0,300,12378923,204,0x40eb3b258a20db0a8145c1089cc614987e5cebc146f4fb60af9b4ab7d9f43084,48,12378991,21,0x6f622a5ff42de9e903318e8aabf039625e1d0db55cf5ac2e522ca9bb86157757,True
130,1.018489e+14,1.018489e+14,-195120.0,-194700.0,0xC36442b4a4522E871399CD717aBDD847Ab11FE88,1.500000e+16,7.317852e+07,1.475283e+16,7.403008e+07,0.000000e+00,2.562171e+03,0.0,0.0,145,12378944,56,0x560cd18f3f3e594fe81451b239208fb39c692b8c84e31400841f92b126f7b4a2,48,12378991,21,0x6f622a5ff42de9e903318e8aabf039625e1d0db55cf5ac2e522ca9bb86157757,True
131,2.222554e+13,2.222554e+13,-195180.0,-194520.0,0xC36442b4a4522E871399CD717aBDD847Ab11FE88,6.586864e+15,2.000000e+07,6.582675e+15,2.001444e+07,0.000000e+00,4.339093e+01,0.0,0.0,328,12378978,124,0x7aeb13b8269ec3a8a7a8c8957e3f0f2ebb5ce861d2b9eaead35be824b7afd66b,48,12378991,21,0x6f622a5ff42de9e903318e8aabf039625e1d0db55cf5ac2e522ca9bb86157757,True
132,2.650137e+07,2.650137e+07,-195000.0,-194760.0,0xC36442b4a4522E871399CD717aBDD847Ab11FE88,2.498902e+09,1.000000e+01,2.493907e+09,1.001721e+01,0.000000e+00,5.173863e-05,0.0,0.0,67,12378979,46,0xca0741b45c497247c61bd0f3ff0fa0bad17160df22d32c46b83c433ff055ef48,48,12378991,21,0x6f622a5ff42de9e903318e8aabf039625e1d0db55cf5ac2e522ca9bb86157757,True


In [128]:
le = clm.view_all_pool_events().iloc[-1]

In [130]:
le['amount']

1.9421012448978996e+16

In [126]:
bpos = pos.loc[(pos['tickLower'] == -196260)&(pos['tickUpper'] == -193380)]
bpos

,last_L,start_L,tickLower,tickUpper,owner,start_token0_holdings,start_token1_holdings,last_token0_holdings,last_token1_holdings,token0_fees_accrued,token1_fees_accrued,token0_collected,token1_collected,start_logIndex,start_blockNumber,start_transactionIndex,start_transactionHash,last_logIndex,last_blockNumber,last_transactionIndex,last_transactionHash,active
64,2.391451e+12,2.391451e+12,-196260.0,-193380.0,0xC36442b4a4522E871399CD717aBDD847Ab11FE88,2.758377e+15,1.000000e+07,2.926148e+15,9.419499e+06,2.501216e+12,6941.856102,0.0,0.0,77,12377638,64,0x8bf4fd993facf998cb4b2579aee65a6053b1ce59a29dc28f6c684c7935226f66,48,12378991,21,0x6f622a5ff42de9e903318e8aabf039625e1d0db55cf5ac2e522ca9bb86157757,True
70,1.066579e+14,1.066579e+14,-196260.0,-193380.0,0xC36442b4a4522E871399CD717aBDD847Ab11FE88,1.261930e+17,4.350007e+08,1.305052e+17,4.201064e+08,1.006686e+14,304931.701869,0.0,0.0,273,12377759,110,0x524a4b69f399fed8655f886865135cb8c1d21913df7c0dad9ed3d7beecefc823,48,12378991,21,0x6f622a5ff42de9e903318e8aabf039625e1d0db55cf5ac2e522ca9bb86157757,True
75,4.900862e+14,4.900862e+14,-196260.0,-193380.0,0xC36442b4a4522E871399CD717aBDD847Ab11FE88,5.554640e+17,2.083474e+09,5.996630e+17,1.930362e+09,4.192342e+14,995941.987740,0.0,0.0,91,12377962,44,0xe4c993872a2173fc890bdd9aaabda0ba6a00ef960e678bc3b8cc8f3f000abcfe,48,12378991,21,0x6f622a5ff42de9e903318e8aabf039625e1d0db55cf5ac2e522ca9bb86157757,True
94,7.060240e+14,7.060240e+14,-196260.0,-193380.0,0xC36442b4a4522E871399CD717aBDD847Ab11FE88,8.260166e+17,2.911791e+09,8.638817e+17,2.780902e+09,3.003979e+14,648515.062354,0.0,0.0,179,12378422,109,0x90f88e6d1d0a65c67bf4bad96f305b0290a1127af6f1750658144e89ce40d9ad,48,12378991,21,0x6f622a5ff42de9e903318e8aabf039625e1d0db55cf5ac2e522ca9bb86157757,True
116,4.240236e+15,4.240236e+15,-196260.0,-193380.0,0xC36442b4a4522E871399CD717aBDD847Ab11FE88,4.789742e+18,1.808250e+10,5.188298e+18,1.670153e+10,1.299647e+15,346764.484617,0.0,0.0,185,12378687,97,0x1ad3ab88fb3a2cb3a1525ae289370f0eab00b67d40675e5430f5b85d64af4d6f,48,12378991,21,0x6f622a5ff42de9e903318e8aabf039625e1d0db55cf5ac2e522ca9bb86157757,True
128,2.538718e+11,2.538718e+11,-196260.0,-193380.0,0xC36442b4a4522E871399CD717aBDD847Ab11FE88,3.106214e+14,1.000000e+06,3.106343e+14,9.999558e+05,2.191726e+09,7.417522,0.0,0.0,252,12378918,54,0xa38513bb1e034de9ec1e6ce6bbd9081a06f0a25145fc89315e66863f3f7e7fcd,48,12378991,21,0x6f622a5ff42de9e903318e8aabf039625e1d0db55cf5ac2e522ca9bb86157757,True
129,2.589468e+16,2.589468e+16,-196260.0,-193380.0,0xC36442b4a4522E871399CD717aBDD847Ab11FE88,3.168309e+19,1.019990e+11,3.168439e+19,1.019945e+11,2.235539e+14,756580.118416,0.0,0.0,300,12378923,204,0x40eb3b258a20db0a8145c1089cc614987e5cebc146f4fb60af9b4ab7d9f43084,48,12378991,21,0x6f622a5ff42de9e903318e8aabf039625e1d0db55cf5ac2e522ca9bb86157757,True


In [132]:
bpos.loc[bpos['last_L'] >= le['amount']] #only one with enough L to count 

,last_L,start_L,tickLower,tickUpper,owner,start_token0_holdings,start_token1_holdings,last_token0_holdings,last_token1_holdings,token0_fees_accrued,token1_fees_accrued,token0_collected,token1_collected,start_logIndex,start_blockNumber,start_transactionIndex,start_transactionHash,last_logIndex,last_blockNumber,last_transactionIndex,last_transactionHash,active
129,2.589468e+16,2.589468e+16,-196260.0,-193380.0,0xC36442b4a4522E871399CD717aBDD847Ab11FE88,3.168309e+19,1.019990e+11,3.168439e+19,1.019945e+11,2.235539e+14,756580.118416,0.0,0.0,300,12378923,204,0x40eb3b258a20db0a8145c1089cc614987e5cebc146f4fb60af9b4ab7d9f43084,48,12378991,21,0x6f622a5ff42de9e903318e8aabf039625e1d0db55cf5ac2e522ca9bb86157757,True


In [87]:
ae.loc[(ae['tickLower'] == -198060)&(ae['tickUpper'] == -192180)]

,event,logIndex,blockNumber,transactionIndex,transactionHash,sender,amount,tickLower,tickUpper,amount0,amount1,recipient,tick,liquidity
168,Mint,299,12376962,196,0xd48ab10b8cb8fb22c033f79bd00b181c471297410cc45ad3699a62a98e8055f4,0xC36442b4a4522E871399CD717aBDD847Ab11FE88,6.608140e+15,-198060.0,-192180.0,1.382461e+19,5.836757e+10,NaN,NaN,NaN
194,Mint,234,12377062,147,0x5e0a0a0f16393e56c2c0caf1fa273429121ee636baab65c700ce75618011f99d,0xC36442b4a4522E871399CD717aBDD847Ab11FE88,7.592098e+13,-198060.0,-192180.0,1.508874e+17,6.982974e+08,NaN,NaN,NaN
207,Burn,288,12377175,217,0x925d59b27727f967fd2554469d8365cbf9aedad895e0df69248d4b79adbc9b84,0xC36442b4a4522E871399CD717aBDD847Ab11FE88,7.592098e+13,-198060.0,-192180.0,1.492770e+17,7.039575e+08,NaN,NaN,NaN


In [39]:
current_tick = clm.tick
active_pos = pos.loc[(pos['tickLower'] < current_tick)&(pos['tickUpper'] >= current_tick)]

In [43]:
int(sum(active_pos['last_L'])) #off by 16 

41448871078369024

In [45]:
int(clm.swaps['liquidity'].iloc[-1])

41448871078369032

In [37]:
clm.view_all_pool_events()

,event,logIndex,blockNumber,transactionIndex,transactionHash,sender,amount,tickLower,tickUpper,amount0,amount1,recipient,tick,liquidity
0,Mint,31,12375326,11,0x2e07c690f149223e4f290986277304ea6a05c6ee47ba303732166bc1b15cbafb,0xC36442b4a4522E871399CD717aBDD847Ab11FE88,8.210905e+15,-197400.0,-193320.0,1.242324e+19,5.050028e+10,NaN,NaN,NaN
1,Mint,307,12375529,212,0x85a3f0ed4fc14085609e95e7366364566b89d7569e62e4a828d1ef1e5dc81e58,0xC36442b4a4522E871399CD717aBDD847Ab11FE88,1.935471e+13,-195120.0,-195060.0,1.000000e+15,0.000000e+00,NaN,NaN,NaN
2,Mint,148,12375553,104,0x0fde2929c1fa5dd5dc9a0fcfbf8adccdeefbc398aa36ff019aa2b15eded23552,0xC36442b4a4522E871399CD717aBDD847Ab11FE88,3.509242e+09,-195180.0,-195120.0,9.995372e+10,2.740000e+02,NaN,NaN,NaN
3,Mint,99,12375566,44,0x34ca11f7f1c6a4d8d17cabf14c6d0ffa5116a693508cd974c882e06308479820,0xC36442b4a4522E871399CD717aBDD847Ab11FE88,3.510779e+11,-195180.0,-195120.0,9.999750e+12,2.741200e+04,NaN,NaN,NaN
4,Mint,105,12375566,45,0x9ece74425a9b5b937b51e14d4fbb9eb7aa6f6ae415d51a460b6f56335d31b9f7,0xC36442b4a4522E871399CD717aBDD847Ab11FE88,3.510779e+11,-195180.0,-195120.0,9.999750e+12,2.741200e+04,NaN,NaN,NaN
5,Swap,207,12375689,195,0x7e348d63a404ee6cd2c400f2c6e730e776c1d25aebe539b788b7c9dcefce842f,0xE592427A0AEce92De3Edee1F18E0157C05861564,NaN,NaN,NaN,1.000000e+15,-3.339863e+06,0xE592427A0AEce92De3Edee1F18E0157C05861564,-195154.0,8.211610e+15
6,Swap,207,12375690,252,0x59a3626c612fc662fa3b626adaafa2358988efcd13a1316bb7c6011f276e5c5e,0xE592427A0AEce92De3Edee1F18E0157C05861564,NaN,NaN,NaN,1.000000e+16,-3.339606e+07,0x4d997c89Bc659a3E8452038A8101161e7e7e53a7,-195155.0,8.211610e+15
7,Swap,79,12375699,111,0x5e213a2e270e51add4e1298562c29cc53ed78f6abf3b7f9678ffe4055722bb46,0xE592427A0AEce92De3Edee1F18E0157C05861564,NaN,NaN,NaN,1.000000e+15,-3.339347e+06,0xbabe61887f1de2713c6f97e567623453d3C79f67,-195155.0,8.211610e+15
8,Swap,352,12375700,235,0xea8b1509ed20b5380530853e984940cec020cad42d7a5d8f0b9709ef809c877b,0xE592427A0AEce92De3Edee1F18E0157C05861564,NaN,NaN,NaN,-1.488183e+16,5.000000e+07,0xE592427A0AEce92De3Edee1F18E0157C05861564,-195153.0,8.211610e+15
9,Swap,353,12375735,202,0x969173d81a5282e5ca583afae3d88efb17fa77b275c00610df9f069f71d87e20,0xE592427A0AEce92De3Edee1F18E0157C05861564,NaN,NaN,NaN,3.000000e+15,-1.001986e+07,0xE4310545E143b6E44Aefa9A96F9aca44984CB1e0,-195154.0,8.211610e+15


In [ ]:
position_df

In [28]:
clm.liquidity

7.416174873917757e+20

In [29]:
clm.get_active_LP_positions()

,last_L,start_L,tickLower,tickUpper,owner,start_token0_holdings,start_token1_holdings,last_token0_holdings,last_token1_holdings,token0_fees_accrued,token1_fees_accrued,token0_collected,token1_collected,start_logIndex,start_blockNumber,start_transactionIndex,start_transactionHash,last_logIndex,last_blockNumber,last_transactionIndex,last_transactionHash,active
2,7.308975e+20,7.308975e+20,-90300.0,-87360.0,0xC36442b4a4522E871399CD717aBDD847Ab11FE88,3.886628e+21,6.814681e+17,9.127638e+21,1.802841e+13,1.882760e+20,2.378577e+16,0.0,0.0,24,12638749,3,0x87eabf5af9ee2d959c32b79d09efd2f0ad91f2f1c6f42f46f4c032e7fef83bbc,12,12957054,1,0x27f9a5cddc4025562be12e9bb74db4f557591667500d0db3e968e648fcd23c23,True
3,1.071994e+19,1.071994e+19,-90300.0,-87360.0,0xC36442b4a4522E871399CD717aBDD847Ab11FE88,9.946499e+18,1.700000e+16,1.338734e+20,2.644195e+11,2.655692e+18,3.114656e+14,0.0,0.0,422,12642203,131,0xcd5ff8e742f15b2a37704f947d16e003b63466c807d00a315b74b21d0fb9167e,12,12957054,1,0x27f9a5cddc4025562be12e9bb74db4f557591667500d0db3e968e648fcd23c23,True


In [30]:
clm.view_all_pool_events()

,event,logIndex,blockNumber,transactionIndex,transactionHash,sender,amount,tickLower,tickUpper,amount0,amount1,recipient,tick,liquidity
0,Mint,191,12504752,115,0x58337bfa434b41a386d33647c0c7d495fdb3b26f4036f52425a9b1bf13c3124b,0xC36442b4a4522E871399CD717aBDD847Ab11FE88,6.805070e+22,-92160.0,-92040.0,2.956435e+22,1.120790e+18,NaN,NaN,NaN
1,Swap,341,12505364,192,0xa65838c86179aac0ca259281eeea1c5039d6e387750a6615c533af4b4aa70da9,0xE592427A0AEce92De3Edee1F18E0157C05861564,NaN,NaN,NaN,-7.990224e+20,8.000000e+16,0x14f967AFcFfbA3F5B16450aA576087a844745174,-92125.0,6.805070e+22
2,Swap,245,12506010,211,0x99b2fb0abf37bd85df7b92a534c1e46d78bb401696dc44566a7a356f4c912bfc,0xc06D57cE9B2AB9155c918AED666c04bFc31Ae020,NaN,NaN,NaN,1.207542e+22,-1.199796e+18,0xc06D57cE9B2AB9155c918AED666c04bFc31Ae020,-92160.0,6.805070e+22
3,Burn,144,12510449,172,0x31d070f1d8f7e4132cac47b09cf0b92a38b37c1096c787e2c1d2a2625d72beaa,0xC36442b4a4522E871399CD717aBDD847Ab11FE88,6.805070e+22,-92160.0,-92040.0,4.080452e+22,7.537319e+14,NaN,NaN,NaN
4,Collect,148,12510449,172,0x31d070f1d8f7e4132cac47b09cf0b92a38b37c1096c787e2c1d2a2625d72beaa,0xC36442b4a4522E871399CD717aBDD847Ab11FE88,NaN,-92160.0,-92040.0,4.084074e+22,9.937319e+14,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
223,Swap,306,12861537,211,0xe9b3b1a8134c704ac4710576c403f116ba66ecaf372190862a6a0c69accccab0,0xE592427A0AEce92De3Edee1F18E0157C05861564,NaN,NaN,NaN,2.313512e+21,-2.865150e+17,0x0000000089341e263B85D84A0Eea39f47C37A9d2,-90286.0,7.416175e+20
224,Swap,55,12917385,3,0x2fff46407dabe6309e82a43a411670a019923844c59de18cb7555e63c89c669c,0xE592427A0AEce92De3Edee1F18E0157C05861564,NaN,NaN,NaN,-2.404288e+21,3.000000e+17,0x5E6732d6416D3f3A6574b61eF1C81316cF8D9287,-89563.0,7.416175e+20
225,Swap,259,12917485,154,0x84021e289ab154bd112cbc83354a0f67e67dbe9e32dcfbaaaef780e0286d2904,0xDef1C0ded9bec7F1a1670819833240f027b25EfF,NaN,NaN,NaN,-3.429817e+20,4.460625e+16,0x74de5d4FCbf63E00296fd95d33236B9794016631,-89458.0,7.416175e+20
226,Swap,77,12920918,30,0xccd6d30929a044a469df9ea41fb0f52ff0bfd4ab75f5517d805ef4e2ae1cc287,0x520d18f8d73F5A6A961dC42834b9291E73731030,NaN,NaN,NaN,2.591638e+21,-3.239186e+17,0x520d18f8d73F5A6A961dC42834b9291E73731030,-90238.0,7.416175e+20


In [31]:
pos = clm.positions

In [32]:
pos

,last_L,start_L,tickLower,tickUpper,owner,start_token0_holdings,start_token1_holdings,last_token0_holdings,last_token1_holdings,token0_fees_accrued,token1_fees_accrued,token0_collected,token1_collected,start_logIndex,start_blockNumber,start_transactionIndex,start_transactionHash,last_logIndex,last_blockNumber,last_transactionIndex,last_transactionHash,active
0,0.000000e+00,6.805070e+22,-92160.0,-92040.0,0xC36442b4a4522E871399CD717aBDD847Ab11FE88,2.956435e+22,1.120790e+18,0.000000e+00,0.000000e+00,3.622626e+19,2.400000e+14,4.084074e+22,9.937319e+14,191,12504752,115,0x58337bfa434b41a386d33647c0c7d495fdb3b26f4036f52425a9b1bf13c3124b,12,12957054,1,0x27f9a5cddc4025562be12e9bb74db4f557591667500d0db3e968e648fcd23c23,True
1,0.000000e+00,3.314221e+20,-95460.0,-82860.0,0xC36442b4a4522E871399CD717aBDD847Ab11FE88,1.235598e+22,5.027857e+17,0.000000e+00,0.000000e+00,1.877827e+20,2.758921e+16,7.216457e+21,1.161534e+18,61,12534909,28,0x47b395a5664558405a7f39fe8d83bf627e613ceba20ef4dbb014c2bdbba2f38d,12,12957054,1,0x27f9a5cddc4025562be12e9bb74db4f557591667500d0db3e968e648fcd23c23,False
2,7.308975e+20,7.308975e+20,-90300.0,-87360.0,0xC36442b4a4522E871399CD717aBDD847Ab11FE88,3.886628e+21,6.814681e+17,9.127638e+21,1.802841e+13,1.882760e+20,2.378577e+16,0.000000e+00,0.000000e+00,24,12638749,3,0x87eabf5af9ee2d959c32b79d09efd2f0ad91f2f1c6f42f46f4c032e7fef83bbc,12,12957054,1,0x27f9a5cddc4025562be12e9bb74db4f557591667500d0db3e968e648fcd23c23,True
3,1.071994e+19,1.071994e+19,-90300.0,-87360.0,0xC36442b4a4522E871399CD717aBDD847Ab11FE88,9.946499e+18,1.700000e+16,1.338734e+20,2.644195e+11,2.655692e+18,3.114656e+14,0.000000e+00,0.000000e+00,422,12642203,131,0xcd5ff8e742f15b2a37704f947d16e003b63466c807d00a315b74b21d0fb9167e,12,12957054,1,0x27f9a5cddc4025562be12e9bb74db4f557591667500d0db3e968e648fcd23c23,True


In [595]:
pos[['last_token0_holdings', 'last_token1_holdings']] = pos.apply(lambda x: clm.get_amounts(clm.sqrtPrice, clm.tick_to_sqrtPrice(x.tickLower), clm.tick_to_sqrtPrice(x.tickUpper), x.last_L), axis = 1, result_type='expand')

In [596]:
pos

,last_L,start_L,tickLower,tickUpper,owner,start_token0_holdings,start_token1_holdings,last_token0_holdings,last_token1_holdings,token0_fees_accrued,token1_fees_accrued,token0_collected,token1_collected,start_logIndex,start_blockNumber,start_transactionIndex,start_transactionHash,last_logIndex,last_blockNumber,last_transactionIndex,last_transactionHash,active
0,0.0,6.805070e+22,-92160.0,-92040.0,0xC36442b4a4522E871399CD717aBDD847Ab11FE88,2.956435e+22,1.120790e+18,0.0,0.0,3.622626e+19,2.400000e+14,4.084074e+22,9.937319e+14,191,12504752,115,0x58337bfa434b41a386d33647c0c7d495fdb3b26f4036f52425a9b1bf13c3124b,148,12510449,172,0x31d070f1d8f7e4132cac47b09cf0b92a38b37c1096c787e2c1d2a2625d72beaa,True


In [593]:
pos['last_token0_holdings'] + pos['token0_fees_accrued'] - pos['token0_collected']

0   -4.080452e+22
dtype: float64

In [594]:
pos['last_token1_holdings'] + pos['token1_fees_accrued'] - pos['token1_collected']

0   -7.537319e+14
dtype: float64

In [537]:
clm.swaps

,event,logIndex,blockNumber,transactionIndex,transactionHash,sender,recipient,amount0,amount1,tick,liquidity
0,Swap,341,12505364,192,0xa65838c86179aac0ca259281eeea1c5039d6e387750a...,0xE592427A0AEce92De3Edee1F18E0157C05861564,0x14f967AFcFfbA3F5B16450aA576087a844745174,-7.990224e+20,8.000000e+16,-92125.0,6.805070e+22
0,Swap,245,12506010,211,0x99b2fb0abf37bd85df7b92a534c1e46d78bb401696dc...,0xc06D57cE9B2AB9155c918AED666c04bFc31Ae020,0xc06D57cE9B2AB9155c918AED666c04bFc31Ae020,1.207542e+22,-1.199796e+18,-92160.0,6.805070e+22
0,Swap,5,12534911,4,0xc6a78c31cc2b2526711e3a058d0fe2f31e2c67d90c6d...,0x0000000000007F150Bd6f54c40A34d7C3d5e9f56,0x0F641eFbEfF4bE14762A276D4d3744cE40526653,-3.021840e+21,3.316864e+17,-90253.0,3.314221e+20
0,Swap,131,12535193,98,0xb4d5e9657c0bfd6bf9a5f2239c54fe0c0b8657e867fb...,0xFD3dFB524B2dA40c8a6D703c62BE36b5D8540626,0xFD3dFB524B2dA40c8a6D703c62BE36b5D8540626,-1.107208e+21,1.387750e+17,-89507.0,3.314221e+20
0,Swap,189,12536446,83,0xb738e264feeafe714f944a9f752249296392ba79d5ba...,0xFD3dFB524B2dA40c8a6D703c62BE36b5D8540626,0xFD3dFB524B2dA40c8a6D703c62BE36b5D8540626,2.690077e+20,-3.447233e+16,-89690.0,3.314221e+20
...,...,...,...,...,...,...,...,...,...,...,...
0,Swap,306,12861537,211,0xe9b3b1a8134c704ac4710576c403f116ba66ecaf3721...,0xE592427A0AEce92De3Edee1F18E0157C05861564,0x0000000089341e263B85D84A0Eea39f47C37A9d2,2.313512e+21,-2.865150e+17,-90286.0,7.416175e+20
0,Swap,55,12917385,3,0x2fff46407dabe6309e82a43a411670a019923844c59d...,0xE592427A0AEce92De3Edee1F18E0157C05861564,0x5E6732d6416D3f3A6574b61eF1C81316cF8D9287,-2.404288e+21,3.000000e+17,-89563.0,7.416175e+20
0,Swap,259,12917485,154,0x84021e289ab154bd112cbc83354a0f67e67dbe9e32dc...,0xDef1C0ded9bec7F1a1670819833240f027b25EfF,0x74de5d4FCbf63E00296fd95d33236B9794016631,-3.429817e+20,4.460625e+16,-89458.0,7.416175e+20
0,Swap,77,12920918,30,0xccd6d30929a044a469df9ea41fb0f52ff0bfd4ab75f5...,0x520d18f8d73F5A6A961dC42834b9291E73731030,0x520d18f8d73F5A6A961dC42834b9291E73731030,2.591638e+21,-3.239186e+17,-90238.0,7.416175e+20


In [473]:
clm.total_fee0, clm.total_fee1 

(0, 0)

In [474]:
tdf.dropna()

event                                                             Swap
logIndex                                                           341
transactionIndex                                                   192
transactionHash      0xa65838c86179aac0ca259281eeea1c5039d6e387750a...
address                     0x537DE9477a639AD1d23Bc2DcfEB8Cc3032bc114C
blockHash            0xf99460710f908f0d58415919cf6f5ebf4618f4363a58...
blockNumber                                                   12505364
args.sqrtPriceX96                        791622562113505313289142272.0
args.tick                                                     -92125.0
args.owner                                                         nan
args.sender                 0xE592427A0AEce92De3Edee1F18E0157C05861564
args.amount0                                  -799022360665044287488.0
args.amount1                                       80000000000000000.0
args.recipient              0x14f967AFcFfbA3F5B16450aA576087a844745174
args.l

In [475]:
blockNumber = tdf['blockNumber']
transactionIndex = tdf['transactionIndex']
logIndex = tdf['logIndex']
transactionHash = tdf['transactionHash']
sender = tdf['args.sender']
recipient = tdf['args.recipient']
amount0 = tdf['args.amount0']
amount1 = tdf['args.amount1']
tick = tdf['args.tick']
liquidity = tdf['args.liquidity']

In [476]:
#Fees for pool 
zeroForOne = None
if amount0 > 0:
    total_fee0 = amount0*clm.fee
    amount0_nf = amount0 - total_fee0
    zeroForOne = True
else:
    total_fee0 = 0
    amount0_nf = amount0

if amount1 > 0:
    total_fee1= amount1*clm.fee
    amount1_nf = amount1 - total_fee1
    zeroForOne = False
else:
    total_fee1 = 0
    amount1_nf = amount1

if zeroForOne == None:
    raise SwapAmountError(f"Swap amounts are incorrect:\n\n\t{(amount0,amount1)}")

clm.total_fee0 += total_fee0
clm.total_fee1 += total_fee1

#can now overright fee for proper adding


In [477]:
pos = clm.positions
current_tick, current_tick_lower, current_tick_upper = clm.get_tick_range(clm.tick)

In [478]:
if zeroForOne:
    sqrtPrice = clm.sqrtPrice 
    sqrtPriceA = clm.tick_to_sqrtPrice(current_tick_lower) #based on lower tick not lower price

    amount0_a = amount0_nf
    amount1_a = amount1_nf

    fee0_collected = 0
    while amount0_a > 0:
        active_pos = pos.loc[(pos['tickLower'] < current_tick)&(pos['tickUpper'] >= current_tick)]
        L = active_pos['last_L'].sum()

        #check if there is enough reserves in the tick
        if clm.get_amount0(sqrtPrice, sqrtPriceA) > amount0_a:
            sqrtPrice_next = clm.get_next_sqrtPrice_from_inputs(sqrtPrice, L, amount0_a, zeroForOne=zeroForOne)
            tick_next = clm.sqrtPrice_to_tick(sqrtPrice_next)

            fee0_in_range = ((amount0_a/(1-clm.fee)) - amount0_a)
            fee0_collected += fee0_in_range
            if not fee0_collected == total_fee1:
                raise FeeMismatch(f"Fee0 collected do not match\n\n\t{fee0_collected}, {total_fee0}")
            
            fee0_per_L = fee0_in_range/L

            pos['token0_fees_accrued'] = pos['token0_fees_accrued'].mask(pos.index.isin(active_pos.index), pos['token0_fees_accrued'] + pos['last_L'] * fee0_per_L)
            break


        else:
            amount0_diff = clm.get_amount0(sqrtPrice, sqrtPriceA, L) 
            amount1_diff = clm.get_amount1(sqrtPrice, sqrtPriceA, L) 

            fee0_in_range = ((amount0_a/(1-clm.fee)) - amount0_a)
            fee0_collected += fee0_in_range
            
            fee0_per_L = fee0_in_range/L
            pos['token0_fees_accrued'] = pos['token0_fees_accrued'].mask(pos.index.isin(active_pos.index), pos['token0_fees_accrued'] + pos['last_L'] * fee0_per_L)

            amount0_a -= amount0_diff
            amount1_a -= amount1_diff
            current_tick = current_tick_lower
            current_tick_lower = current_tick - clm.tickSpacing
            sqrtPrice = clm.tick_to_sqrtPrice(current_tick)
            sqrtPriceA = clm.tick_to_sqrtPrice(current_tick_lower)


In [448]:
#Under amount
if not any([liquidity, tick]): #save if check not given
    clm.sqrtPrice = sqrtPrice_next
    clm.tick = tick_next
    clm.liquidity = L

else:
    if (L != liquidity) or (tick != tick_next):
        raise SwapAllignmentError(f"Swap: tick or liquidity provided does not match calculations\n\n\tliquidity: {L}, {liquidity}\n\ttick: {tick}, {tick_next}")
    else:
        clm.sqrtPrice = sqrtPrice_next
        clm.tick = tick_next
        clm.liquidity = L

In [479]:
if not zeroForOne:#else:
    sqrtPrice = clm.sqrtPrice
    sqrtPriceB = clm.tick_to_sqrtPrice(current_tick_upper) #based on upper tick not upper price

    amount0_a = amount0_nf
    amount1_a = amount1_nf

    fee1_collected = 0
    while amount1_a > 0:
        active_pos = pos.loc[(pos['tickLower'] <= current_tick)&(pos['tickUpper'] > current_tick)]
        L = active_pos['last_L'].sum()

        #check if there is enough reserves in the tick
        if clm.get_amount1(sqrtPrice, sqrtPriceB, L) > amount1_a:
            sqrtPrice_next = clm.get_next_sqrtPrice_from_inputs(sqrtPrice, L, amount1_a, zeroForOne=zeroForOne)
            tick_next = clm.sqrtPrice_to_tick(sqrtPrice_next)

            fee1_in_range = ((amount1_a/(1-clm.fee)) - amount1_a)
            fee1_collected += fee1_in_range
            if not fee1_collected == total_fee1:
                raise FeeMismatch(f"Fee1 collected do not match\n\n\t{fee1_collected}, {total_fee1}")
            
            fee1_per_L = fee1_in_range/L

            pos['token1_fees_accrued'] = pos['token1_fees_accrued'].mask(pos.index.isin(active_pos.index), pos['token1_fees_accrued'] + pos['last_L'] * fee1_per_L)
            break

        else:
            amount0_diff = clm.get_amount0(sqrtPrice, sqrtPriceB, L) 
            amount1_diff = clm.get_amount1(sqrtPrice, sqrtPriceB, L) 

            fee1_in_range = ((amount1_diff/(1-clm.fee)) - amount1_diff)
            fee1_collected += fee1_in_range
            fee1_per_L = fee1_in_range/L
            pos['token1_fees_accrued'] = pos['token1_fees_accrued'].mask(pos.index.isin(active_pos.index), pos['token1_fees_accrued'] + pos['last_L'] * fee1_per_L)

            amount0_a -= amount0_diff
            amount1_a -= amount0_diff

            current_tick = current_tick_upper
            current_tick_upper = current_tick + clm.tickSpacing
            sqrtPrice = clm.tick_to_sqrtPrice(current_tick)
            sqrtPriceB = clm.tick_to_sqrtPrice(current_tick_upper)


In [480]:
amount1_a

7.976e+16

In [482]:
pos['token1_fees_accrued']

0    2.400000e+14
Name: token1_fees_accrued, dtype: float64

In [462]:
fee1_in_range = ((amount1_a/(1-clm.fee)) - amount1_a)

In [463]:
fee1_per_L = fee1_in_range/L

In [464]:
fee1_per_L * L

240000000000000.0

In [465]:
pos['token1_fees_accrued'] = pos['token1_fees_accrued'].mask(pos.index.isin(active_pos.index), pos['token1_fees_accrued'] + pos['last_L'] * fee1_per_L)

In [458]:
pos

,last_L,start_L,tickLower,tickUpper,owner,start_token0_holdings,start_token1_holdings,last_token0_holdings,last_token1_holdings,token0_fees_accrued,...,token1_collected,start_logIndex,start_blockNumber,start_transactionIndex,start_transactionHash,last_logIndex,last_blockNumber,last_transactionIndex,last_transactionHash,active
0,6.805070e+22,6.805070e+22,-92160.0,-92040.0,0xC36442b4a4522E871399CD717aBDD847Ab11FE88,2.956435e+22,1.120790e+18,2.956435e+22,1.120790e+18,0.0,...,0.0,191,12504752,115,0x58337bfa434b41a386d33647c0c7d495fdb3b26f4036...,191,12504752,115,0x58337bfa434b41a386d33647c0c7d495fdb3b26f4036...,True


In [376]:
sqrtPrice_next = clm.get_next_sqrtPrice_from_inputs(sqrtPrice, L, amount1_a, zeroForOne=zeroForOne)

0.009991681455075785

In [377]:
clm.get_amount1(sqrtPrice, sqrtPriceB, L) 

9.183864200783355e+17

In [378]:
amount1_a

7.976e+16

In [343]:
pos

,last_L,start_L,tickLower,tickUpper,owner,start_token0_holdings,start_token1_holdings,last_token0_holdings,last_token1_holdings,token0_fees_accrued,...,token1_collected,start_logIndex,start_blockNumber,start_transactionIndex,start_transactionHash,last_logIndex,last_blockNumber,last_transactionIndex,last_transactionHash,active
0,6.805070e+22,6.805070e+22,-92160.0,-92040.0,0xC36442b4a4522E871399CD717aBDD847Ab11FE88,2.956435e+22,1.120790e+18,2.956435e+22,1.120790e+18,0.0,...,0.0,191,12504752,115,0x58337bfa434b41a386d33647c0c7d495fdb3b26f4036...,191,12504752,115,0x58337bfa434b41a386d33647c0c7d495fdb3b26f4036...,True


In [ ]:
        #Fees for pool 
        if amount0 > 0:
            total_fee0 = amount0*clm.fee
            amount0_nf = amount0 - total_fee0
        else:
            total_fee0 = 0
            amount0_nf = amount0

        if amount1 > 0:
            total_fee1= amount1*clm.fee
            amount1_nf = amount1 - total_fee1
        else:
            total_fee1 = 0
            amount1_nf = amount1

        clm.total_fee0 += total_fee0
        clm.total_fee1 += total_fee1

        pos = clm.positions
        current_tick, current_tick_lower, current_tick_upper = clm.get_tick_range(clm.tick)

        if amount0_nf > 0:
            sqrtPrice_current = clm.sqrtPrice 
            sqrtPriceA = clm.tick_to_sqrtPrice(current_tick_lower) #based on lower tick not lower price

            amount0_a = amount0_nf
            amount1_a = amount1_nf

            while amount0_a > 0:
                apos = pos.loc[(pos['tickLower'] < current_tick)&(pos['tickUpper'] >= current_tick)]
                L_tick = apos['last_L'].sum()

                #check if there is enough reserves in the tick
                if L_tick*(1/sqrtPriceA - 1/sqrtPrice_current) > amount0_a:
                    sqrtPrice_new = 1/((amount0_a/L_tick)+(1/sqrtPrice_current))
                    tick_new = clm.sqrtPrice_to_tick(sqrtPrice_new)
                    break
                    #TODO Add fee to apos

                else:
                    amount0_a -= L_tick *(1/sqrtPriceA - 1/sqrtPrice_current)
                    amount1_a -= L_tick *(sqrtPriceA - sqrtPrice_current)
                    current_tick = current_tick_lower
                    current_tick_lower = current_tick - clm.tickSpacing
                    sqrtPrice_current = clm.tick_to_sqrtPrice(current_tick)
                    sqrtPriceA = clm.tick_to_sqrtPrice(current_tick_lower)
                    #TODO Add fee to apos
            
            if not any([liquidity, tick]): #save if check not given
                clm.sqrtPrice = sqrtPrice_new
                clm.tick = tick_new
                clm.liquidity = L_tick

            else:
                if (L_tick != liquidity) or (tick != tick_new):
                    raise SwapAllignmentError(f"Swap: tick or liquidity provided does not match calculations\n\n\thave liquidity: {L_tick}, {liquidity}\n\ttick: {tick}, {tick_new}")
                else:
                    clm.sqrtPrice = sqrtPrice_new
                    clm.tick = tick_new
                    clm.liquidity = L_tick

        
        if amount1_nf > 0:
            sqrtPrice_current = clm.sqrtPrice
            sqrtPriceB = clm.tick_to_sqrtPrice(current_tick_upper) #based on upper tick not upper price

            amount0_a = amount0_nf
            amount1_a = amount1_nf

            while amount1_a > 0:
                apos = pos.loc[(pos['tickLower'] <= current_tick)&(pos['tickUpper'] > current_tick)]
                L_tick = apos['last_L'].sum()

                #check if there is enough reserves in the tick
                if L_tick*(sqrtPriceB - sqrtPrice_current) > amount1_a:
                    sqrtPrice_new = sqrtPrice_current + (amount1_a/L_tick)
                    tick_new = clm.sqrtPrice_to_tick(sqrtPrice_new)
                    #TODO add fee to apos
                    break

                else:
                    amount0_a -= L_tick *(1/sqrtPriceB - 1/sqrtPrice_current)
                    amount1_a -= L_tick *(sqrtPriceB - sqrtPrice_current)

                    #TODO add fee to apos
                    current_tick = current_tick_upper
                    current_tick_upper = current_tick + clm.tickSpacing
                    sqrtPrice_current = clm.tick_to_sqrtPrice(current_tick)
                    sqrtPriceB = clm.tick_to_sqrtPrice(current_tick_upper)
            
            if not any([liquidity, tick]): #save if check not given
                clm.sqrtPrice = sqrtPrice_new
                clm.tick = tick_new
                clm.liquidity = L_tick

            else:
                if (L_tick != liquidity) or (tick != tick_new):
                    raise SwapAllignmentError(f"Swap: tick or liquidity provided does not match calculations\n\n\thave liquidity: {L_tick}, {liquidity}\n\ttick: {tick}, {tick_new}")
                else:
                    clm.sqrtPrice = sqrtPrice_new
                    clm.tick = tick_new
                    clm.liquidity = L_tick
        

In [252]:
clm.burns

,event,logIndex,blockNumber,transactionIndex,transactionHash,sender,amount,tickLower,tickUpper,amount0,amount1
0,Burn,144,12510449,172,0x31d070f1d8f7e4132cac47b09cf0b92a38b37c1096c7...,0xC36442b4a4522E871399CD717aBDD847Ab11FE88,6.805070e+22,-92160.0,-92040.0,4.080452e+22,7.537319e+14


In [253]:
tickLower = tdf['args.tickLower']#, 
tickUpper = tdf['args.tickUpper']#, 
amount0 = tdf['args.amount0']#,
amount1 = tdf['args.amount1']#,
recipient = tdf['args.recipient']#,
blockNumber = tdf['blockNumber']#, 
transactionIndex = tdf['transactionIndex']#, 
logIndex = tdf['logIndex']#, 
transactionHash = tdf['transactionHash']

In [254]:
collect_df = pd.DataFrame([['Collect', logIndex, blockNumber, transactionIndex, transactionHash, recipient, tickLower, tickUpper, amount0, amount1]], 
                       columns=['event', 'logIndex', 'blockNumber', 'transactionIndex', 'transactionHash', 'sender', 'tickLower', 'tickUpper', 'amount0', 'amount1'])
collect_df

,event,logIndex,blockNumber,transactionIndex,transactionHash,sender,tickLower,tickUpper,amount0,amount1
0,Collect,148,12510449,172,0x31d070f1d8f7e4132cac47b09cf0b92a38b37c1096c7...,0xC36442b4a4522E871399CD717aBDD847Ab11FE88,-92160.0,-92040.0,4.084074e+22,9.937319e+14


In [255]:
pos = clm.positions.reset_index(drop=True) #can maybe maintain index through the mints?
#check if len >1, otherwise burn liquidity in pool
cpos = pos.loc[(pos['tickLower'] == tickLower) & (pos['tickUpper'] == tickUpper)]


In [256]:
cpos

,last_L,start_L,tickLower,tickUpper,owner,start_token0_holdings,start_token1_holdings,last_token0_holdings,last_token1_holdings,token0_fees_accrued,...,token1_collected,start_logIndex,start_blockNumber,start_transactionIndex,start_transactionHash,last_logIndex,last_blockNumber,last_transactionIndex,last_transactionHash,active
0,0.0,6.805070e+22,-92160.0,-92040.0,0xC36442b4a4522E871399CD717aBDD847Ab11FE88,2.956435e+22,1.120790e+18,4.080452e+22,7.537319e+14,0.0,...,0.0,191,12504752,115,0x58337bfa434b41a386d33647c0c7d495fdb3b26f4036...,144,12510449,172,0x31d070f1d8f7e4132cac47b09cf0b92a38b37c1096c7...,True


In [257]:
if len(cpos) > 1:
    #TODO handle cases when bounds do not match
    clpos = cpos.loc[(cpos['L'] == 0)&((cpos['last_token0_holdings'] > 0)|(cpos['last_token1_holdings'] > 0))] 
    if len(clpos) == 1:
        i = clpos.index[0]

    elif len(clpos) == 0:
        raise CollectMatchError("Cannot match mints and burns because there is no null L in same tick range")

    else:
        raise CollectMatchError("Cannot match mints and burns more the one position with same L")

else:
    i = cpos.index[0]

In [258]:
int(amount1)

993731921441930

In [259]:
#pos['token0_collected'] = pos['token0_collected'].astype(float)
#pos['token1_collected'] = pos['token1_collected'].astype(float)

In [260]:
pos['token0_collected'].iat[i] += amount0
pos['token1_collected'].iat[i] += amount1


In [261]:
pos

,last_L,start_L,tickLower,tickUpper,owner,start_token0_holdings,start_token1_holdings,last_token0_holdings,last_token1_holdings,token0_fees_accrued,...,token1_collected,start_logIndex,start_blockNumber,start_transactionIndex,start_transactionHash,last_logIndex,last_blockNumber,last_transactionIndex,last_transactionHash,active
0,0.0,6.805070e+22,-92160.0,-92040.0,0xC36442b4a4522E871399CD717aBDD847Ab11FE88,2.956435e+22,1.120790e+18,4.080452e+22,7.537319e+14,0.0,...,9.937319e+14,191,12504752,115,0x58337bfa434b41a386d33647c0c7d495fdb3b26f4036...,144,12510449,172,0x31d070f1d8f7e4132cac47b09cf0b92a38b37c1096c7...,True


In [262]:
#if == 0 then position is now inactive with nothing to collect 
#if can then update the last holdings to be negative 

In [263]:
token0_diff = (pos['last_token0_holdings'].iloc[i] + pos['token0_fees_accrued'].iloc[i]  - pos['token0_collected'].iloc[i])

In [264]:
token1_diff = (pos['last_token1_holdings'].iloc[i] + pos['token1_fees_accrued'].iloc[i]  - pos['token1_collected'].iloc[i])

In [269]:
if (token0_diff <= 0) & (token1_diff <= 0): #Should really be ==
    pos['last_token0_holdings'].iat[i] = 0
    pos['last_token1_holdings'].iat[i] = 0
    pos['active'].iat[i] = False

In [271]:
pos['last_token1_holdings'] + pos['token1_fees_accrued'] - pos['token1_collected']

0   -9.937319e+14
dtype: float64

In [81]:
def get_amount0_for_liquidity(sqrtPriceA, sqrtPriceB, liquidity):
    if sqrtPriceA > sqrtPriceB:
        sqrtPriceA, sqrtPriceB = sqrtPriceB, sqrtPriceA
    
    return liquidity * ((1/sqrtPriceA) - (1/sqrtPriceB))

def get_amount1_for_liquidity(sqrtPriceA, sqrtPriceB, liquidity):
    if sqrtPriceA > sqrtPriceB:
        sqrtPriceA, sqrtPriceB = sqrtPriceB, sqrtPriceA
    
    return (liquidity * (sqrtPriceB - sqrtPriceA))


def get_amounts_for_liquidity(sqrtPrice, sqrtPriceA, sqrtPriceB, liquidity):
    amount0 = 0
    amount1 = 0

    if sqrtPriceA > sqrtPriceB:
        sqrtPriceA, sqrtPriceB = sqrtPriceB, sqrtPriceA
    
    if sqrtPrice <= sqrtPriceA:
        amount0 = get_amount0_for_liquidity(sqrtPriceA, sqrtPriceB, liquidity)
    
    elif sqrtPrice < sqrtPriceB:
        amount0 = get_amount0_for_liquidity(sqrtPrice, sqrtPriceB, liquidity)
        amount1 = get_amount1_for_liquidity(sqrtPriceA, sqrtPrice, liquidity)
    
    else:
        amount1 = get_amount1_for_liquidity(sqrtPriceA, sqrtPriceB, liquidity)

    return amount0, amount1

In [82]:
clm.tick_to_sqrtPrice(tickUpper), clm.tick_to_sqrtPrice(tickLower)

(0.010034060581277992, 0.009974039462207726)

In [83]:
(cpos['start_L'] - cpos['last_L']).values[0]

6.805070034957683e+22

In [84]:
camount0, camount1 = get_amounts_for_liquidity(clm.sqrtPrice, clm.tick_to_sqrtPrice(tickUpper), clm.tick_to_sqrtPrice(tickLower), (cpos['start_L'] - cpos['last_L']).values[0])

In [86]:
camount0, camount1 

(4.080451818908354e+22, 753731577068398.1)

In [87]:
amount0, amount1

(4.084074444134624e+22, 993731921441930.0)

In [85]:
camount0 - amount0 

-3.622625226270127e+19

In [80]:
camount1 - amount1

-240000344373531.88

In [ ]:
#simply add that fees are collected #may result in a error between portfolio cost
#use burn to reallign the 

In [74]:
pos['L'].at[i]  -= bpos['L'] 

KeyError: 'L'

In [30]:
pos

,L,tickLower,tickUpper,owner,start_token0_holdings,start_token1_holdings,last_token0_holdings,last_token1_holdings,token0_fees_accrued,token1_fees_accrued,token0_fees_collected,token1_fees_collected,start_logIndex,start_blockNumber,start_transactionIndex,start_transactionHash,last_logIndex,last_blockNumber,last_transactionIndex,last_transactionHash
0,0.0,-92160.0,-92040.0,0xC36442b4a4522E871399CD717aBDD847Ab11FE88,2.956435e+22,1.120790e+18,2.956435e+22,1.120790e+18,0,0,0,0,191,12504752,115,0x58337bfa434b41a386d33647c0c7d495fdb3b26f4036...,144,12510449,172,0x31d070f1d8f7e4132cac47b09cf0b92a38b37c1096c7...


In [330]:
#add q96 for precision?
#in code it is 0 rounding up and 1 rounding down 
#tick has been fixed

In [444]:
tdf.dropna()

event                                                            Burn
logIndex                                                          144
transactionIndex                                                  172
transactionHash     0x31d070f1d8f7e4132cac47b09cf0b92a38b37c1096c7...
address                    0x537DE9477a639AD1d23Bc2DcfEB8Cc3032bc114C
blockHash           0x19ea521d6960d4139fad5c7eed49f059241ad0d1d896...
blockNumber                                                  12510449
args.owner                 0xC36442b4a4522E871399CD717aBDD847Ab11FE88
args.tickLower                                               -92160.0
args.tickUpper                                               -92040.0
args.sender                                                       nan
args.amount                                 68050700349576828682240.0
args.amount0                                40804518185646702985216.0
args.amount1                                        753731921441931.0
args.recipient      

In [450]:
tickLower = tdf['args.tickLower']#, 
tickUpper = tdf['args.tickUpper']#, 
amount = tdf['args.amount']#,
amount0 = tdf['args.amount0']#,
amount1 = tdf['args.amount1']#,
owner = tdf['args.owner']#,
blockNumber = tdf['blockNumber']#, 
transactionIndex = tdf['transactionIndex']#, 
logIndex = tdf['logIndex']#, 
transactionHash = tdf['transactionHash']

In [475]:
pos = clm.positions.reset_index(drop=True)

In [476]:
#check if len >1, otherwise burn liquidity in pool
mpos = pos.loc[(pos['tickLower'] == tickLower) & (pos['tickUpper'] == tickUpper)]

In [477]:
if len(mpos) > 1:
    print('need to match by owner')
    #check the liquidity amounts, if more then 1 equals fifo
    #if none equals then fifo for partial, this may be incorrect but best without sender being correct


In [489]:
i = mpos.index[0]

In [490]:
i

0

In [493]:
pos['L'].at[i]  -= mpos['L']

In [495]:
pos

,L,tickLower,tickUpper,owner,start_token0_holdings,start_token1_holdings,last_token0_holdings,last_token1_holdings,token0_fees_accrued,token1_fees_accrued,token0_fees_collected,token1_fees_collected,start_logIndex,start_blockNumber,start_transactionIndex,start_transactionHash,last_logIndex,last_blockNumber,last_transactionIndex,last_transactionHash
0,0.0,-92160.0,-92040.0,0xC36442b4a4522E871399CD717aBDD847Ab11FE88,2.956435e+22,1.120790e+18,2.956435e+22,1.120790e+18,0,0,0,0,191,12504752,115,0x58337bfa434b41a386d33647c0c7d495fdb3b26f4036...,191,12504752,115,0x58337bfa434b41a386d33647c0c7d495fdb3b26f4036...


In [502]:
def position_last_update_state(position, blockNumber, transactionIndex, logIndex, transactionHash):
    #will update all positions without needing to drop duplicates
    
    position['last_blockNumber'] = blockNumber
    position['last_transactionIndex'] = transactionIndex
    position['last_logIndex'] = logIndex
    position['last_transactionHash'] = transactionHash

    return position

In [505]:
pos = position_last_update_state(pos, blockNumber, transactionIndex, logIndex, transactionHash)

In [506]:
#active pos = pos

,L,tickLower,tickUpper,owner,start_token0_holdings,start_token1_holdings,last_token0_holdings,last_token1_holdings,token0_fees_accrued,token1_fees_accrued,token0_fees_collected,token1_fees_collected,start_logIndex,start_blockNumber,start_transactionIndex,start_transactionHash,last_logIndex,last_blockNumber,last_transactionIndex,last_transactionHash
0,0.0,-92160.0,-92040.0,0xC36442b4a4522E871399CD717aBDD847Ab11FE88,2.956435e+22,1.120790e+18,2.956435e+22,1.120790e+18,0,0,0,0,191,12504752,115,0x58337bfa434b41a386d33647c0c7d495fdb3b26f4036...,144,12510449,172,0x31d070f1d8f7e4132cac47b09cf0b92a38b37c1096c7...


In [507]:
burn_df = pd.DataFrame([['Burn', logIndex, blockNumber, transactionIndex, transactionHash, owner, amount, tickLower, tickUpper, amount0, amount1]], 
                       columns=['event', 'logIndex', 'blockNumber', 'transactionIndex', 'transactionHash', 'sender', 'amount', 'tickLower', 'tickUpper', 'amount0', 'amount1'])
        

In [508]:
burn_df

,event,logIndex,blockNumber,transactionIndex,transactionHash,sender,amount,tickLower,tickUpper,amount0,amount1
0,Burn,144,12510449,172,0x31d070f1d8f7e4132cac47b09cf0b92a38b37c1096c7...,0xC36442b4a4522E871399CD717aBDD847Ab11FE88,6.805070e+22,-92160.0,-92040.0,4.080452e+22,7.537319e+14


In [511]:
mpos.loc[mpos['L'] == amount]

,L,tickLower,tickUpper,owner,start_token0_holdings,start_token1_holdings,last_token0_holdings,last_token1_holdings,token0_fees_accrued,token1_fees_accrued,token0_fees_collected,token1_fees_collected,start_logIndex,start_blockNumber,start_transactionIndex,start_transactionHash,last_logIndex,last_blockNumber,last_transactionIndex,last_transactionHash
0,6.805070e+22,-92160.0,-92040.0,0xC36442b4a4522E871399CD717aBDD847Ab11FE88,2.956435e+22,1.120790e+18,2.956435e+22,1.120790e+18,0,0,0,0,191,12504752,115,0x58337bfa434b41a386d33647c0c7d495fdb3b26f4036...,144,12510449,172,0x31d070f1d8f7e4132cac47b09cf0b92a38b37c1096c7...


In [331]:
def getNextSqrtPriceX96FromAmount0RoundingUp(sqrtPriceX96, L, amonutIn): 
    return int((L * Q96 * sqrtPriceX96) // (L* Q96 + amonutIn * sqrtPriceX96))

In [332]:
def getNextSqrtPriceX96FromAmount1RoundingDown(sqrtPriceX96, L, amonutIn): 
    return sqrtPriceX96 + (amonutIn * Q96 // L)

In [333]:
def getNextSqrtPriceFromInput(sqrtPrice, L, amonutIn, zeroForOne):
    if zeroForOne:
        sqrtPriceNext = getNextSqrtPriceX96FromAmount0RoundingUp(sqrtPrice, L, amonutIn)
    else:
        sqrtPriceNext = getNextSqrtPriceX96FromAmount1RoundingDown(sqrtPrice, L, amonutIn)
    return sqrtPriceNext

In [334]:
#if 0 for 1 then 

In [335]:
#Get L
def calc_liquidity_from_amount0(amount, sqrtPrice_lower, sqrtPrice_upper):
    if sqrtPrice_lower > sqrtPrice_upper:
        sqrtPrice_lower, sqrtPrice_upper = sqrtPrice_upper, sqrtPrice_lower
    return (amount * (sqrtPrice_lower * sqrtPrice_upper)/Q96) / (sqrtPrice_upper - sqrtPrice_lower)

def calc_liquidity_from_amount1(amount, sqrtPrice_lower, sqrtPrice_upper):
    if sqrtPrice_upper > sqrtPrice_lower:
        sqrtPrice_upper, sqrtPrice_lower = sqrtPrice_lower, sqrtPrice_upper
    return amount * Q96 / (sqrtPrice_lower - sqrtPrice_upper)

def calc_liquidity_from_amounts(amount0, amount1, sqrtPriceX96, tickLower, tickUpper):
    sqrtPrice_lower = clm.tick_to_sqrtPrice(tickLower) * Q96
    sqrtPrice_upper = clm.tick_to_sqrtPrice(tickUpper) * Q96

    liquidity0 = calc_liquidity_from_amount0(amount0, sqrtPriceX96, sqrtPrice_upper)
    liquidity1 = calc_liquidity_from_amount1(amount1, sqrtPriceX96, sqrtPrice_lower)

    #L = min(liquidity0, liquidity1)

    return int(liquidity0), int(liquidity1)


In [336]:
#with fees track the amount of fee per L in pool to scale easily 

In [337]:
def sqrtPrice_to_tick(sqrtPrice):
    return math.floor(round(math.log(sqrtPrice, math.sqrt(1.0001)), 6)) #control for precision issues and tick int size with the rounding

In [338]:
def tick_to_sqrtPrice(tick):
    return  (1.0001 ** (tick / 2))

In [339]:
#init
TICK_SPACING = 60
FEE = 3000*10**-6

sqrtPriceX96 = df['args.sqrtPriceX96'].iloc[0]
tick = int(df['args.tick'].iloc[0])

sqrtPriceX96, tick, FEE, TICK_SPACING

(7.915297013757073e+26, -92127, 0.003, 60)

In [340]:
sqrtPrice_to_tick(sqrtPriceX96/Q96), tick

(-92127, -92127)

In [341]:
sqrtPrice_to_tick(sqrtPriceX96/Q96) == tick #check if tick is correct

True

In [342]:
#mint
L = df['args.amount'].iloc[1]
amount0_mint = df['args.amount0'].iloc[1]
amount1_mint = df['args.amount1'].iloc[1]
tickUpper = df['args.tickUpper'].iloc[1]
tickLower = df['args.tickLower'].iloc[1]

L, tickLower, tickUpper, amount0_mint, amount1_mint

(6.805070034957683e+22,
 -92160.0,
 -92040.0,
 2.9564348235497557e+22,
 1.120789970578036e+18)

In [344]:
#check if the L is correct
sqrtPrice_lower = tick_to_sqrtPrice(tickLower)
sqrtPrice_upper = tick_to_sqrtPrice(tickUpper)


In [345]:
sqrtPrice_lower, sqrtPrice_upper

(0.009974039462207726, 0.010034060581277992, 0.009974039462207726)

In [346]:
sqrtPrice_to_tick(sqrtPrice_lower), sqrtPrice_to_tick(sqrtPrice_upper)

(-92160, -92040)

In [347]:
int(L)

68050700349576828682240

In [348]:
L0, L1 = calc_liquidity_from_amounts(amount0_mint, amount1_mint, sqrtPriceX96, tickLower, tickUpper)

In [349]:
L0, L1

(68050700341664718782464, 68050700370488672124928)

In [350]:
min(L0, L1)

68050700341664718782464

In [351]:
int(L) - L0 

7912109899776

In [352]:
int(L) - L1

-20911843442688

In [239]:
L

6.805070034957683e+22

In [240]:
#try calc next price from amounts

In [137]:
amount0_mint, amount1_mint, sqrtPrice_lower, sqrtPriceX96/Q96, sqrtPrice_upper

(2.4999999999999993e+20,
 2721739239788589.0,
 0.3161803726108171,
 0.3162119906480381,
 3.8726239653714614)

In [143]:
int(amount1_mint/((sqrtPriceX96/Q96) - sqrtPrice_lower))

86081853239756816384

In [232]:
int(amount1_mint/((sqrtPriceX96/Q96) - sqrtPrice_lower)) - int(L) #higher then current

20911843442688

In [233]:
int(amount0_mint/((1/(sqrtPriceX96/Q96)) - (1/(sqrtPrice_upper))))

68050700341664492290048

In [231]:
int(amount0_mint/((1/(sqrtPriceX96/Q96)) - (1/sqrtPrice_upper))) - int(L)

-7912336392192

In [152]:
getNextSqrtPriceX96FromAmount0RoundingUp(sqrtPriceX96,L,amount0_mint)

13059627426680280554233921536

In [410]:
cam0, cam1 = calc_amounts(sqrtPriceX96, sqrtPrice_lower*Q96, sqrtPrice_upper*Q96, L)
cam0, cam1 

(921461136512239061404279472390144, 7894069736660084088373248)

In [411]:
amount0_mint, amount1_mint

(2.9564348235497557e+22, 1.120789970578036e+18)

In [412]:
amount0_mint - cam0

-9.214611364826747e+32

In [413]:
amount1_mint - cam1

-7.894068615870114e+24

In [371]:
tick_to_sqrtPrice(sqrtPrice_to_tick(sqrtPriceX96/Q96))

0.00999050938778999

In [382]:
ac0, ac1 = get_amounts_for_liquidity(tick_to_sqrtPrice(sqrtPrice_to_tick(sqrtPriceX96/Q96)), sqrtPrice_lower, sqrtPrice_upper, L)

In [383]:
amount0_mint - ac0

16848519168.0

In [384]:
amount1_mint - ac1

-405120.0

In [372]:
#L amounts being calculated do not line up with what is being supplied here

In [210]:
#swap
amount0 = df['args.amount0'].iloc[2]
amount1 = df['args.amount1'].iloc[2]
tick_Supplied = int(df['args.tick'].iloc[2])
liquidity = df['args.liquidity'].iloc[2]
sqrtPriceX96_Supplied = df['args.sqrtPriceX96'].iloc[2]

amount0, amount1, tick_Supplied, liquidity, sqrtPriceX96_Supplied

(-4.907891792174419e+18,
 5e+17,
 -22665,
 8.608185313068412e+19,
 2.551285573970388e+28)

In [179]:
sqrtPrice_to_tick(sqrtPriceX96_Supplied/Q96) == tick_Supplied

True

In [420]:
def calc_amount0(L, sqrtPriceX96_A, sqrtPriceX96_B):
    #sqrtPriceX96
    if sqrtPriceX96_A > sqrtPriceX96_B:
        sqrtPriceX96_A, sqrtPriceX96_B = sqrtPriceX96_B, sqrtPriceX96_A
    return int(((L * Q96 * (sqrtPriceX96_B - sqrtPriceX96_A)) / sqrtPriceX96_B) / sqrtPriceX96_A)


def calc_amount1(L, sqrtPriceX96_A, sqrtPriceX96_B):
    #sqrtPriceX96
    if sqrtPriceX96_A > sqrtPriceX96_B:
        sqrtPriceX96_A, sqrtPriceX96_B = sqrtPriceX96_B, sqrtPriceX96_A
    return int(L * (sqrtPriceX96_B - sqrtPriceX96_A) / Q96)

In [421]:
def calc_amounts(sqrtRatioX96, sqrtRatioAX96, sqrtRatioBX96, liquidity):
    amount0 = 0
    amount1 = 0

    if sqrtRatioAX96 > sqrtRatioBX96:
        sqrtRatioAX96, sqrtRatioBX96 = sqrtRatioBX96, sqrtRatioAX96
    
    if sqrtRatioX96 <= sqrtRatioAX96:
        amount0 = calc_amount0(sqrtRatioAX96, sqrtRatioBX96, liquidity)
    
    elif sqrtRatioX96 < sqrtRatioBX96:
        amount0 = calc_amount0(sqrtRatioX96, sqrtRatioBX96, liquidity)
        amount1 = calc_amount1(sqrtRatioAX96, sqrtRatioX96, liquidity)
    
    else:
        amount1 = calc_amount1(sqrtRatioAX96, sqrtRatioBX96, liquidity)

    return amount0, amount1


In [424]:
calc_amounts(sqrtPriceX96, sqrtPrice_upper*Q96, sqrtPrice_lower*Q96, L)

(921461136512239061404279472390144, 7894069736660084088373248)

In [ ]:
#check which way it needs to go and calc to target
def compute_swap_step(sqrtPriceCurrentX96, sqrtPriceTargetX96, L, amountRemaining):
    zeroForOne = True if sqrtPriceCurrentX96 >= sqrtPriceTargetX96 else False
    sqrtPriceNextX96 = getNextSqrtPriceFromInput(sqrtPriceCurrentX96, L, amountRemaining, zeroForOne)

    amountIn = calc_amount0(L, sqrtPriceCurrentX96, sqrtPriceTargetX96)
    amountOut = calc_amount1(L, sqrtPriceCurrentX96, sqrtPriceTargetX96)

    if not zeroForOne:
        amountIn, amountOut = amountOut, amountIn

    return sqrtPriceNextX96, amountIn, amountOut

In [177]:
clm.active_positions

,L,tickLower,tickUpper,owner,start_token0_holdings,start_token1_holdings,last_token0_holdings,last_token1_holdings,token0_fees_accrued,token1_fees_accrued,token0_fees_collected,token1_fees_collected,start_logIndex,start_blockNumber,start_transactionIndex,start_transactionHash,last_logIndex,last_blockNumber,last_transactionIndex,last_transactionHash
0,8.608185e+19,-23030.0,27080.0,0xC36442b4a4522E871399CD717aBDD847Ab11FE88,2.500000e+20,2.721739e+15,2.500000e+20,2.721739e+15,0,0,0,0,41,12518239,53,0xa4a62f4e9c90a5e98ad0bae083f7caeb9eddf2019441...,41,12518239,53,0xa4a62f4e9c90a5e98ad0bae083f7caeb9eddf2019441...


In [178]:
tdf.dropna()

event                                                             Swap
logIndex                                                            55
transactionIndex                                                     9
transactionHash      0x5cbbaac966f33c4c41709e7aea7fe94115fb322cf699...
address                     0x500Dcb12218a66b4Aca4cc246Af1E4003D02E042
blockHash            0xee9490a00de83ca7af0cba34e8b17b7b6d86b346fc99...
blockNumber                                                   12518312
args.sqrtPriceX96                      25512855739703879120160030720.0
args.tick                                                     -22665.0
args.owner                                                         nan
args.sender                 0xE592427A0AEce92De3Edee1F18E0157C05861564
args.amount0                                    -4907891792174418944.0
args.amount1                                      500000000000000000.0
args.recipient              0xfD9ffc51175278Dd0Bc1de8AA8Fe8065b0CB90C0
args.l

In [33]:
#With the burns,
#FIFO if cannot trust the id
#Ideally it would be from the nft transfers and match the nft to transaction logs
#First match range, then address, if no address fifo


In [34]:
blockNumber = tdf['blockNumber']
transactionIndex = tdf['transactionIndex']
logIndex = tdf['logIndex']
transactionHash = tdf['transactionHash']
sender = tdf['args.sender']
recipient = tdf['args.recipient']
amount0 = tdf['args.amount0']
amount1 = tdf['args.amount1']
tick = tdf['args.tick']
liquidity = tdf['args.liquidity']




In [46]:
swap_df = pd.DataFrame([['Swap', logIndex, blockNumber, transactionIndex, transactionHash, sender, recipient, amount0, amount1, tick, liquidity]], 
                       columns=['event', 'logIndex', 'blockNumber', 'transactionIndex', 'transactionHash', 'sender', 'recipient', 'amount0', 'amount1', 'tick', 'liquidity'])
swap_df

,event,logIndex,blockNumber,transactionIndex,transactionHash,sender,recipient,amount0,amount1,tick,liquidity
0,Swap,55,12518312,9,0x5cbbaac966f33c4c41709e7aea7fe94115fb322cf699...,0xE592427A0AEce92De3Edee1F18E0157C05861564,0xfD9ffc51175278Dd0Bc1de8AA8Fe8065b0CB90C0,-4.907892e+18,5.000000e+17,-22665.0,8.608185e+19


In [47]:
#Fees for pool 
if amount0 > 0:
    total_fee0 = int(amount0*clm.fee)
    amount0_nf = amount0 - total_fee0
else:
    total_fee0 = 0
    amount0_nf = amount0


if amount1 > 0:
    total_fee1 = int(amount1*clm.fee)
    amount1_nf = amount1 - total_fee1
else:
    total_fee1 = 0
    amount1_nf = amount1


In [48]:
total_fee0, total_fee1 #+= on total fees #fees needed to be rounded cant have part of a token, i think it is down/truncated

(0, 250000000000000)

In [49]:
#check active positions in tick if no L move to next tick, 
#if 1 is being added we go up the tick 
#if 0 is being added we go down the tick 


In [50]:
if amount0_nf > 0:
    print('going up the tick')

if amount1_nf > 0:
    print('going down the tick')

going down the tick


In [51]:
amount0_nf, amount1_nf

(-4.907891792174419e+18, 4.9975e+17)

In [52]:
#calc of new price based on dy
#sqrtPrice_current - (amount1_nf/L_tick) 

In [53]:
pos = clm.active_positions
pos

,L,tickLower,tickUpper,owner,start_token0_holdings,start_token1_holdings,last_token0_holdings,last_token1_holdings,token0_fees_accrued,token1_fees_accrued,token0_fees_collected,token1_fees_collected,start_logIndex,start_blockNumber,start_transactionIndex,start_transactionHash,last_logIndex,last_blockNumber,last_transactionIndex,last_transactionHash
0,8.608185e+19,-23030.0,27080.0,0xC36442b4a4522E871399CD717aBDD847Ab11FE88,2.500000e+20,2.721739e+15,2.500000e+20,2.721739e+15,0,0,0,0,41,12518239,53,0xa4a62f4e9c90a5e98ad0bae083f7caeb9eddf2019441...,41,12518239,53,0xa4a62f4e9c90a5e98ad0bae083f7caeb9eddf2019441...


In [54]:
swap_df

,event,logIndex,blockNumber,transactionIndex,transactionHash,sender,recipient,amount0,amount1,tick,liquidity
0,Swap,55,12518312,9,0x5cbbaac966f33c4c41709e7aea7fe94115fb322cf699...,0xE592427A0AEce92De3Edee1F18E0157C05861564,0xfD9ffc51175278Dd0Bc1de8AA8Fe8065b0CB90C0,-4.907892e+18,5.000000e+17,-22665.0,8.608185e+19


In [55]:
current_tick, current_tick_lower, current_tick_upper = clm.get_tick_range(clm.tick)

In [56]:
current_tick

-23028.0

In [57]:
#loop from here

In [58]:
apos = pos.loc[(pos['tickLower'] < current_tick)&(pos['tickUpper'] >= current_tick)]
L_tick = apos['L'].astype(float).sum()

In [59]:
clm.sqrtPrice, L_tick

(0.3162119906480381, 8.608185313068412e+19)

In [60]:
swap_df['liquidity'] == L_tick 

0    True
Name: liquidity, dtype: bool

In [61]:
amount0_nf, amount1_nf

(-4.907891792174419e+18, 4.9975e+17)

In [62]:
sqrtPriceX96 = clm.sqrtPrice * clm.Q96

In [80]:
sqrtPrice_next = getNextSqrtPriceFromInput(sqrtPriceX96, float(L_tick), amount0, zeroForOne=True)/Q96
sqrtPrice_next

0.3220175115775343

In [81]:
sqrtPrice_next = getNextSqrtPriceFromInput(clm.sqrtPrice*Q96, float(L_tick), amount1, zeroForOne=False)/Q96
sqrtPrice_next

0.3220204157901053

In [82]:
tick_next = clm.sqrtPrice_to_tick(sqrtPrice_next) #correct
L_next = L_tick 

In [83]:
tick_next, L_next #is tick one off ?

(-22664, 8.608185313068412e+19)

In [85]:
sqrtPrice_next

0.3220204157901053

In [87]:
swap_df['transactionHash'].values

array(['0x5cbbaac966f33c4c41709e7aea7fe94115fb322cf699c2c05d849140d1cc524c'],
      dtype=object)

In [89]:
tsqrtP = clm.tick_to_sqrtPrice(-22665)

In [96]:
int(amount0)

-4907891792174418944

In [97]:
calc_amount0(L_tick, sqrtPriceX96, sqrtPrice_next*Q96)

4910302680603624448

In [79]:
#get amounts 0 and 1 to check

In [90]:
def calc_amount0(L, sqrtPriceX96_A, sqrtPriceX96_B):
    #sqrtPriceX96
    if sqrtPriceX96_A > sqrtPriceX96_B:
        sqrtPriceX96_A, sqrtPriceX96_B = sqrtPriceX96_B, sqrtPriceX96_A
    return int(L * Q96 * (sqrtPriceX96_B - sqrtPriceX96_A) / sqrtPriceX96_B / sqrtPriceX96_A)


def calc_amount1(L, sqrtPriceX96_A, sqrtPriceX96_B):
    #sqrtPriceX96
    if sqrtPriceX96_A > sqrtPriceX96_B:
        sqrtPriceX96_A, sqrtPriceX96_B = sqrtPriceX96_B, sqrtPriceX96_A
    return int(L * (sqrtPriceX96_B - sqrtPriceX96_A) / Q96)

In [355]:
camount0 = calc_amount0(L_tick, sqrtPrice_next*Q96, sqrtPriceX96)
camount0

4907891792174426112

In [356]:
int(amount0_nf) 

-4907891792174418944

In [357]:
(camount0 - int(amount0_nf))/camount0

1.9999999999999984

In [358]:
camount1 = calc_amount1(L_tick, sqrtPrice_next*Q96, sqrtPriceX96)
camount1

499750000000000768

In [359]:
int(amount1_nf) 

499750000000000000

In [360]:
(camount1 - int(amount1_nf)) 

768

In [361]:
calc_amount1(L_tick, sqrtPriceX96, sqrtPrice_next*Q96)

499750000000000768

In [375]:
def compute_swap_step(sqrtPriceCurrentX96, sqrtPriceTargetX96, L, amountRemaining):
    zeroForOne = True if sqrtPriceCurrentX96 <= sqrtPriceTargetX96 else False
    sqrtPriceNextX96 = getNextSqrtPriceFromInput(sqrtPriceCurrentX96, L, amountRemaining, zeroForOne)

    amountIn = calc_amount0(L, sqrtPriceCurrentX96, sqrtPriceTargetX96)
    amountOut = calc_amount1(L, sqrtPriceCurrentX96, sqrtPriceTargetX96)

    if not zeroForOne:
        amountIn, amountOut = amountOut, amountIn

    return sqrtPriceNextX96, amountIn, amountOut

In [376]:
amount0

-4.907891792174419e+18

In [380]:
sqrtPriceNextX96, amountIn, amountOut = compute_swap_step(sqrtPriceX96, sqrtPrice_next, L_tick, amount1)

In [381]:
clm.sqrtPrice_to_tick(sqrtPriceNextX96/Q96)

-22664

In [388]:
math.log(((sqrtPriceNextX96/Q96)**2),1.0001)

-22663.939768905293

In [390]:
math.floor(math.log(((sqrtPriceNextX96/Q96)**2),1.0001)

-22663

In [382]:
swap_df

,event,logIndex,blockNumber,transactionIndex,transactionHash,sender,recipient,amount0,amount1,tick,liquidity
0,Swap,55,12518312,9,0x5cbbaac966f33c4c41709e7aea7fe94115fb322cf699...,0xE592427A0AEce92De3Edee1F18E0157C05861564,0xfD9ffc51175278Dd0Bc1de8AA8Fe8065b0CB90C0,-4.907892e+18,5.000000e+17,-22665.0,8.608185e+19


In [367]:
#for mints to check correct

In [368]:
def calcLiquidityFromAmount0(amount, sqrtPrice_lower, sqrtPrice_upper):
    if sqrtPrice_lower > sqrtPrice_upper:
        sqrtPrice_lower, sqrtPrice_upper = sqrtPrice_upper, sqrtPrice_lower
    return (amount * (sqrtPrice_lower * sqrtPrice_upper)/Q96 ) / (sqrtPrice_upper - sqrtPrice_lower)

In [369]:
def calcLiquidityFromAmount1(amount, sqrtPrice_lower, sqrtPrice_upper):
    if sqrtPrice_upper > sqrtPrice_lower:
        sqrtPrice_upper, sqrtPrice_lower = sqrtPrice_lower, sqrtPrice_upper
    return amount * Q96 / (sqrtPrice_lower - sqrtPrice_upper)

In [164]:
def calc_liquidity_from_amounts(amount0, amount1, sqrtPrice):
    current_tick, current_tick_lower, current_tick_upper = clm.get_tick_range(clm.sqrtPrice_to_tick(sqrtPrice))
    sqrtPrice_lower = clm.tick_to_sqrtPrice(current_tick_lower)
    sqrtPrice_upper = clm.tick_to_sqrtPrice(current_tick_upper)

    liquidity0 = calcLiquidityFromAmount0(amount0, sqrtPrice, sqrtPrice_upper)
    liquidity1 = calcLiquidityFromAmount1(amount1, sqrtPrice, sqrtPrice_lower)

    L = math.floor(min(liquidity0, liquidity1)) 

    return L

In [26]:
pos = clm.active_positions
current_tick, current_tick_lower, current_tick_upper = clm.get_tick_range(clm.tick)

if amount0_nf > 0:
    sqrtPrice_current = clm.sqrtPrice
    sqrtPrice_lower = clm.tick_to_sqrtPrice(current_tick_lower)

    amount0_a = amount0_nf
    amount1_a = amount1_nf

    while amount0_a > 0:
        apos = pos.loc[(pos['tickLower'] < current_tick)&(pos['tickUpper'] >= current_tick)]
        L_tick = apos['L'].sum()

        #check if there is enough reserves in the tick
        if L_tick*(1/sqrtPrice_lower - 1/sqrtPrice_current) > amount0_a:
            sqrtPrice_new = 1/((amount0_a/L_tick)+(1/sqrtPrice_current))
            tick_new = clm.sqrtPrice_to_tick(sqrtPrice_new)
            break
            #TODO Add fee to apos

        else:
            amount0_a -= L_tick *(1/sqrtPrice_lower - 1/sqrtPrice_current)
            amount1_a -= L_tick *(sqrtPrice_lower - sqrtPrice_current)
            current_tick = current_tick_lower
            current_tick_lower = current_tick - clm.tickSpacing
            sqrtPrice_current = clm.tick_to_sqrtPrice(current_tick)
            sqrtPrice_lower = clm.tick_to_sqrtPrice(current_tick_lower)
            #TODO Add fee to apos

In [27]:
sqrtPrice_new

NameError: name 'sqrtPrice_new' is not defined

In [28]:
tick_new

NameError: name 'tick_new' is not defined

In [29]:
swap_df['liquidity'].values[0]

7.531849207101876e+23

In [30]:
L_tick - swap_df['liquidity'].values[0] #off by a few bips, Why? 

NameError: name 'L_tick' is not defined

In [74]:
apos['L'].sum()

2.3788573596325747e+21

In [23]:
current_tick, current_tick_lower, current_tick_upper = clm.get_tick_range(clm.tick)

#Pick the side up or down the range

sqrtPrice_current = clm.sqrtPrice
sqrtPrice_lower = clm.tick_to_sqrtPrice(current_tick_lower)

amount0_a = amount0_nf
amount1_a = amount1_nf

In [24]:
amount0_a, amount1_a

(8.81851364409421e+19, -6.479182651067172e+17)

In [25]:
apos = pos.loc[(pos['tickLower'] < current_tick)&(pos['tickUpper'] >= current_tick)]
L_tick = apos['L'].sum()
apos

,L,tickLower,tickUpper,owner,start_token0_holdings,start_token1_holdings,last_token0_holdings,last_token1_holdings,token0_fees_accrued,token1_fees_accrued,token0_fees_collected,token1_fees_collected,start_logIndex,start_blockNumber,start_transactionIndex,start_transactionHash,last_logIndex,last_blockNumber,last_transactionIndex,last_transactionHash
0,1.836801e+21,-50520.0,-47160.0,0xC36442b4a4522E871399CD717aBDD847Ab11FE88,2.087676e+21,1.000000e+19,2.087676e+21,1.000000e+19,0,0,0,0,283,12376506,173,0x6deb67cb498010802d532dbe2b50cb70bf56c20b1937...,283,12376506,173,0x6deb67cb498010802d532dbe2b50cb70bf56c20b1937...
0,2.567967e+20,-51300.0,-47100.0,0xC36442b4a4522E871399CD717aBDD847Ab11FE88,3.000000e+20,2.183696e+18,3.000000e+20,2.183696e+18,0,0,0,0,184,12376622,69,0xa2a1c2540b51c685464188d8fbc8101cc41e803bf50c...,184,12376622,69,0xa2a1c2540b51c685464188d8fbc8101cc41e803bf50c...


In [26]:
L_tick *(1/sqrtPrice_lower - 1/sqrtPrice_current)

4.686759973719647e+19

In [27]:
amount0_a - L_tick *(1/sqrtPrice_lower - 1/sqrtPrice_current)

4.131753670374563e+19

In [31]:
current_tick, current_tick_lower, current_tick_upper = clm.get_tick_range(clm.tick)

#Pick the side up or down the range

sqrtPrice_current = clm.sqrtPrice
sqrtPrice_lower = clm.tick_to_sqrtPrice(current_tick_lower)

amount0_a = amount0_nf
amount1_a = amount1_nf

apos = pos.loc[(pos['tickLower'] < current_tick)&(pos['tickUpper'] >= current_tick)]
L_tick = apos['L'].sum()

while amount0_a > 0:
    apos = pos.loc[(pos['tickLower'] < current_tick)&(pos['tickUpper'] >= current_tick)]
    L_tick = apos['L'].sum()

    #check if there is enough reserves in the tick
    if L_tick*(1/sqrtPrice_lower - 1/sqrtPrice_current) > amount0_a:
        sqrtPrice_new = 1/((amount0_a/L_tick)+(1/sqrtPrice_current))
        tick_new = clm.sqrtPrice_to_tick(sqrtPrice_new)
        break
        #TODO Add fee to apos

    else:
        amount0_a -= L_tick *(1/sqrtPrice_lower - 1/sqrtPrice_current)
        amount1_a -= L_tick *(sqrtPrice_lower - sqrtPrice_current)
        current_tick = current_tick_lower
        current_tick_lower = current_tick - clm.tickSpacing
        sqrtPrice_current = clm.tick_to_sqrtPrice(current_tick)
        sqrtPrice_lower = clm.tick_to_sqrtPrice(current_tick_lower)
        #TODO Add fee to apos

In [34]:
amount0_nf, amount1_nf 

(-1.7237222177362402e+20, 1.724685375e+21)

In [211]:
L_tick*(1/sqrtPrice_lower - 1/sqrtPrice_current) 

4.686759973719647e+19

In [212]:
amount0_a

8.81851364409421e+19

In [205]:
#check if there is enough reserves in the tick
if L_tick*(1/sqrtPrice_lower - 1/sqrtPrice_current) > amount0_a:
    sqrtPrice_new = 1/((amount0_a/L_tick)+(1/sqrtPrice_current))
    tick_new = clm.sqrtPrice_to_tick(sqrtPrice_new)
else:
    amount0_a -= L_tick *(1/sqrtPrice_lower - 1/sqrtPrice_current)
    amount1_a -= L_tick *(sqrtPrice_lower - sqrtPrice_current)

    current_tick = current_tick_lower
    current_tick_lower = current_tick - clm.tickSpacing
    sqrtPrice_current = clm.tick_to_sqrtPrice(current_tick)
    sqrtPrice_upper = clm.tick_to_sqrtPrice(current_tick_upper)

    



False

In [213]:
amount0_a -= L_tick *(1/sqrtPrice_lower - 1/sqrtPrice_current)
amount1_a -= L_tick *(sqrtPrice_lower - sqrtPrice_current)

In [214]:
amount0_a, amount1_a

(4.131753670374563e+19, -3.030205321024859e+17)

In [215]:
current_tick = current_tick_lower
current_tick_lower = current_tick - clm.tickSpacing
sqrtPrice_current = clm.tick_to_sqrtPrice(current_tick)
sqrtPrice_upper = clm.tick_to_sqrtPrice(current_tick_upper)

In [216]:
sqrtPrice_current, current_tick

(0.08570216782513962, -49140)

In [35]:
current_tick, current_tick_lower, current_tick_upper = clm.get_tick_range(clm.tick)

#Pick the side up or down the range

sqrtPrice_current = clm.sqrtPrice
sqrtPrice_upper = clm.tick_to_sqrtPrice(current_tick_upper)

amount0_a = amount0_nf
amount1_a = amount1_nf

while amount1_a > 0:
    apos = pos.loc[(pos['tickLower'] <= current_tick)&(pos['tickUpper'] > current_tick)]
    L_tick = apos['L'].sum()

    #check if there is enough reserves in the tick
    if L_tick*(sqrtPrice_upper - sqrtPrice_current) > amount1_a:
        sqrtPrice_new = sqrtPrice_current + (amount1_a/L_tick) #could be wrong 
        tick_new = clm.sqrtPrice_to_tick(sqrtPrice_new)
        #TODO add fee to apos
        #TODO save state
        break

    else:
        amount0_a -= L_tick *(1/sqrtPrice_upper - 1/sqrtPrice_current)
        amount1_a -= L_tick *(sqrtPrice_upper - sqrtPrice_current)

        #TODO add fee to apos
        current_tick = current_tick_upper
        current_tick_upper = current_tick + clm.tickSpacing
        sqrtPrice_current = clm.tick_to_sqrtPrice(current_tick)
        sqrtPrice_upper = clm.tick_to_sqrtPrice(current_tick_upper)


In [36]:
L_tick*(sqrtPrice_upper - sqrtPrice_current)

1.7451709863160457e+21

In [37]:
amount1_a 

1.724685375e+21

In [38]:
if L_tick*(sqrtPrice_upper - sqrtPrice_current) > amount1_a:
    print('liq in tick')

liq in tick


In [39]:
damount_0 = L_tick *(1/sqrtPrice_upper - 1/sqrtPrice_current)
damount_1 = L_tick *(sqrtPrice_upper - sqrtPrice_current)

In [40]:
damount_0, damount_1

(-1.7441813967523722e+20, 1.7451709863160457e+21)

In [41]:
amount0_a, amount1_a

(-1.7237222177362402e+20, 1.724685375e+21)

In [42]:
(sqrtPrice_upper - sqrtPrice_current)

0.002317055132583512

In [43]:
current_tick, current_tick_lower, current_tick_upper = clm.get_tick_range(clm.tick)

#Pick the side up or down the range

sqrtPrice_current = clm.sqrtPrice
sqrtPrice_upper = clm.tick_to_sqrtPrice(current_tick_upper)

amount0_a = amount0_nf
amount1_a = amount1_nf

In [44]:
amount0_a, amount1_a

(-1.7237222177362402e+20, 1.724685375e+21)

In [45]:
apos = pos.loc[(pos['tickLower'] < current_tick)&(pos['tickUpper'] >= current_tick)]
print(len(apos))
L_tick = apos['L'].sum()

1


In [46]:
(L_tick*(sqrtPrice_current - sqrtPrice_upper))


-1.7451709863160457e+21

In [47]:
L_tick

7.531849207101876e+23

In [48]:
sqrtPrice_new, clm.sqrtPrice_to_tick(sqrtPrice_new)

(3.164306158759981, 23040)

In [49]:
current_tick, current_tick_lower, current_tick_upper = clm.get_tick_range(clm.tick)

#Pick the side up or down the range

sqrtPrice_current = clm.sqrtPrice
sqrtPrice_lower = clm.tick_to_sqrtPrice(current_tick_lower)

amount0_a = amount0_nf
amount1_a = amount1_nf

while amount1_a > 0:
    apos = pos.loc[(pos['tickLower'] < current_tick)&(pos['tickUpper'] >= current_tick)]
    print(len(apos))
    L_tick = apos['L'].sum()

    #check if there is enough reserves in the tick
    if ((L_tick*(sqrtPrice_current - sqrtPrice_lower)) - amount1_a) > 0:
        sqrtPrice_new = sqrtPrice_current - (amount1_nf/L_tick) 
        tick_new = clm.sqrtPrice_to_tick(sqrtPrice_new)
        print(sqrtPrice_new, tick_new)
        #TODO add fee to apos
        break
    else:
        amount0_a -= L_tick *(1/sqrtPrice_current - 1/sqrtPrice_lower)
        amount1_a -= L_tick *(sqrtPrice_current - sqrtPrice_lower)
        print(amount0_a, amount1_a)
        #TODO add fee to apos
        current_tick = current_tick_lower
        current_tick_lower = current_tick -  clm.tickSpacing
        sqrtPrice_current = clm.tick_to_sqrtPrice(current_tick)
        sqrtPrice_lower = clm.tick_to_sqrtPrice(current_tick_lower)
        print(sqrtPrice_current, current_tick)
        

1
3.159726445790226 23011


In [667]:
#loop from here

In [609]:
sqrtPrice_current = clm.sqrtPrice
sqrtPrice_lower = clm.tick_to_sqrtPrice(current_tick_lower)

In [610]:
amount1_a -= apos['L'].sum()*(sqrtPrice_current - sqrtPrice_lower)

In [611]:
amount0_a -= apos['L'].sum()*(1/sqrtPrice_current - 1/sqrtPrice_lower)

In [612]:
amount0_a, amount1_a

(-2.794087835187154e+19, 2.110177210959484e+17)

In [613]:
#new i

In [614]:
current_tick = current_tick_lower
current_tick_lower = current_tick -  clm.tickSpacing

In [615]:
sqrtPrice_current = clm.tick_to_sqrtPrice(current_tick)
sqrtPrice_lower = clm.tick_to_sqrtPrice(current_tick_lower)

In [616]:
current_tick, current_tick_lower

(-49260.0, -49320.0)

In [617]:
sqrtPrice_current, sqrtPrice_lower

(0.0851895199317025, 0.08493434708098344)

In [618]:
apos = pos.loc[(pos['tickLower'] < current_tick)&(pos['tickUpper'] >= current_tick)]
apos

,L,tickLower,tickUpper,owner,start_token0_holdings,start_token1_holdings,last_token0_holdings,last_token1_holdings,token0_fees_accrued,token1_fees_accrued,token0_fees_collected,token1_fees_collected,start_logIndex,start_blockNumber,start_transactionIndex,start_transactionHash,last_logIndex,last_blockNumber,last_transactionIndex,last_transactionHash
0,1.836801e+21,-50520.0,-47160.0,0xC36442b4a4522E871399CD717aBDD847Ab11FE88,2.087676e+21,1.000000e+19,2.087676e+21,1.000000e+19,0,0,0,0,283,12376506,173,0x6deb67cb498010802d532dbe2b50cb70bf56c20b1937...,283,12376506,173,0x6deb67cb498010802d532dbe2b50cb70bf56c20b1937...
0,2.567967e+20,-51300.0,-47100.0,0xC36442b4a4522E871399CD717aBDD847Ab11FE88,3.000000e+20,2.183696e+18,3.000000e+20,2.183696e+18,0,0,0,0,184,12376622,69,0xa2a1c2540b51c685464188d8fbc8101cc41e803bf50c...,184,12376622,69,0xa2a1c2540b51c685464188d8fbc8101cc41e803bf50c...


In [622]:
tick_L = apos['L'].sum()

In [624]:
if tick_L*(sqrtPrice_current - sqrtPrice_lower) > amount0_nf:
    

True

In [619]:
amount1_a -= apos['L'].sum()*(sqrtPrice_current - sqrtPrice_lower)

In [620]:
amount0_a -= apos['L'].sum()*(1/sqrtPrice_current - 1/sqrtPrice_lower)

In [621]:
amount0_a, amount1_a # sign has fliped, gone to far

(4.589341605296516e+19, -3.232115161390132e+17)

In [ ]:
#first check there is enough in range 
#then calc the new sqrtP
#check the sqrtP with the log and also the liquidity

In [ ]:
tick_L*(((tick_L*sqrtPrice_current)/((dxf*sqrtPrice_current) + tick_L))-sqrtPrice_current)

In [ ]:
dy += L*(((L*sqrtP)/((dx*sqrtP) + L))-sqrtP)

NameError: name 'dy' is not defined

In [ ]:
#fee is proportion in bound of total in by L weight 

In [486]:
amount0_a/amount0_nf

0.20554488914502483

In [393]:
pos.loc[(pos['tickLower']  < current_tick)&(pos['tickUpper'] > current_tick)]

,L,tickLower,tickUpper,owner,start_token0_holdings,start_token1_holdings,last_token0_holdings,last_token1_holdings,token0_fees_accrued,token1_fees_accrued,token0_fees_collected,token1_fees_collected,start_logIndex,start_blockNumber,start_transactionIndex,start_transactionHash,last_logIndex,last_blockNumber,last_transactionIndex,last_transactionHash
0,1.139244e+21,-49260.0,-49200.0,0xC36442b4a4522E871399CD717aBDD847Ab11FE88,2.000000e+18,2.769779e+17,2.000000e+18,2.769779e+17,0,0,0,0,209,12376332,84,0xa3a37265f2f428b110d89a0df297a47b537f77b9cceb...,209,12376332,84,0xa3a37265f2f428b110d89a0df297a47b537f77b9cceb...
0,1.836801e+21,-50520.0,-47160.0,0xC36442b4a4522E871399CD717aBDD847Ab11FE88,2.087676e+21,1.000000e+19,2.087676e+21,1.000000e+19,0,0,0,0,283,12376506,173,0x6deb67cb498010802d532dbe2b50cb70bf56c20b1937...,283,12376506,173,0x6deb67cb498010802d532dbe2b50cb70bf56c20b1937...
0,2.567967e+20,-51300.0,-47100.0,0xC36442b4a4522E871399CD717aBDD847Ab11FE88,3.000000e+20,2.183696e+18,3.000000e+20,2.183696e+18,0,0,0,0,184,12376622,69,0xa2a1c2540b51c685464188d8fbc8101cc41e803bf50c...,184,12376622,69,0xa2a1c2540b51c685464188d8fbc8101cc41e803bf50c...


In [394]:
current_tick - -49200

-3.0

In [ ]:
swap_df = pd.DataFrame([['Burn', logIndex, blockNumber, transactionIndex, transactionHash, sender, amount, tickLower, tickUpper, amount0, amount1]], 
                       columns=['event', 'logIndex', 'blockNumber', 'transactionIndex', 'transactionHash', 'sender', 'amount', 'tickLower', 'tickUpper', 'amount0', 'amount1'])
        

In [231]:
clm.mints

,logIndex,blockNumber,transactionIndex,transactionHash,sender,amount,tickLower,tickUpper,amount0,amount1
0,209,12376332,84,0xa3a37265f2f428b110d89a0df297a47b537f77b9cceb...,0xC36442b4a4522E871399CD717aBDD847Ab11FE88,1.139244e+21,-49260.0,-49200.0,2.000000e+18,2.769779e+17


In [232]:
clm.view_all_pool_events()

,logIndex,blockNumber,transactionIndex,transactionHash,sender,amount,tickLower,tickUpper,amount0,amount1
0,209,12376332,84,0xa3a37265f2f428b110d89a0df297a47b537f77b9cceb...,0xC36442b4a4522E871399CD717aBDD847Ab11FE88,1.139244e+21,-49260.0,-49200.0,2.000000e+18,2.769779e+17


In [233]:
clm.active_positions

,L,tickLower,tickUpper,owner,start_token0_holdings,start_token1_holdings,last_token0_holdings,last_token1_holdings,token0_fees_accrued,token1_fees_accrued,token0_fees_collected,token1_fees_collected,start_logIndex,start_blockNumber,start_transactionIndex,start_transactionHash,last_logIndex,last_blockNumber,last_transactionIndex,last_transactionHash
0,1.139244e+21,-49260.0,-49200.0,0xC36442b4a4522E871399CD717aBDD847Ab11FE88,2.000000e+18,2.769779e+17,2.000000e+18,2.769779e+17,0,0,0,0,209,12376332,84,0xa3a37265f2f428b110d89a0df297a47b537f77b9cceb...,209,12376332,84,0xa3a37265f2f428b110d89a0df297a47b537f77b9cceb...


In [3]:
df = pd.read_parquet("/Volumes/luke-uni/data/uniswap-v3/raw/0x157dfa656fdf0d18e1ba94075a53600d81cb3a97/0x157dfa656fdf0d18e1ba94075a53600d81cb3a97_12000000_13000000.parquet")

In [5]:
df

,event,logIndex,transactionIndex,transactionHash,address,blockHash,blockNumber,args.sqrtPriceX96,args.tick,args.owner,args.tickLower,args.tickUpper,args.sender,args.amount,args.amount0,args.amount1,args.recipient,args.liquidity,args.observationCardinalityNextOld,args.observationCardinalityNextNew
0,Initialize,204,84,0xa3a37265f2f428b110d89a0df297a47b537f77b9cceb...,0x157Dfa656Fdf0D18E1bA94075a53600D81cB3a97,0xa6f91c98be020ba125040ac0a9e4b7c4cc8bc2888d76...,12376332,6.768671e+27,-49203.0,None,NaN,NaN,None,NaN,NaN,NaN,None,NaN,NaN,NaN
1,Mint,209,84,0xa3a37265f2f428b110d89a0df297a47b537f77b9cceb...,0x157Dfa656Fdf0D18E1bA94075a53600D81cB3a97,0xa6f91c98be020ba125040ac0a9e4b7c4cc8bc2888d76...,12376332,NaN,NaN,0xC36442b4a4522E871399CD717aBDD847Ab11FE88,-49260.0,-49200.0,0xC36442b4a4522E871399CD717aBDD847Ab11FE88,1.139244e+21,2.000000e+18,2.769779e+17,None,NaN,NaN,NaN
2,Mint,227,155,0x863f5e65702bc42bbbb1cf35c4ad304157f35db193eb...,0x157Dfa656Fdf0D18E1bA94075a53600D81cB3a97,0xfb834e338e4231ba632e0bbacaf338978323568bba04...,12376372,NaN,NaN,0xC36442b4a4522E871399CD717aBDD847Ab11FE88,-48300.0,-39120.0,0xC36442b4a4522E871399CD717aBDD847Ab11FE88,7.284870e+18,3.000000e+19,0.000000e+00,None,NaN,NaN,NaN
3,Mint,128,102,0x05e81227d86a62ea8612c26c50b334af1035b6de686e...,0x157Dfa656Fdf0D18E1bA94075a53600D81cB3a97,0x2c67a5586170f12849237c04dd1b5ae388b9d015c099...,12376379,NaN,NaN,0xC36442b4a4522E871399CD717aBDD847Ab11FE88,-49200.0,-49140.0,0xC36442b4a4522E871399CD717aBDD847Ab11FE88,2.852599e+20,1.000000e+19,0.000000e+00,None,NaN,NaN,NaN
4,Mint,283,173,0x6deb67cb498010802d532dbe2b50cb70bf56c20b1937...,0x157Dfa656Fdf0D18E1bA94075a53600D81cB3a97,0xf88bf563561b6d12f620941d987a852860c6faa45880...,12376506,NaN,NaN,0xC36442b4a4522E871399CD717aBDD847Ab11FE88,-50520.0,-47160.0,0xC36442b4a4522E871399CD717aBDD847Ab11FE88,1.836801e+21,2.087676e+21,1.000000e+19,None,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2283,Swap,27,57,0x9e654e87735851fa4dad4dc4c6f5c8a3a5f20dce80b8...,0x157Dfa656Fdf0D18E1bA94075a53600D81cB3a97,0x5b6366f3c613999523537199b078db15b091f7acb602...,12997586,4.743888e+27,-56313.0,None,NaN,NaN,0x0000008CF69d25162321FEd9f6789f2A5caDE6bC,NaN,-6.568876e+20,2.356900e+18,0x0000006daea1723962647b7e189d311d757Fb793,1.769061e+22,NaN,NaN
2284,Swap,20,2,0xe3189011db80787152e561f8ef0303215d030a10dc33...,0x157Dfa656Fdf0D18E1bA94075a53600D81cB3a97,0x74e747717d56ed65795b069374b95e614c562d2f4215...,12997731,4.715904e+27,-56431.0,None,NaN,NaN,0xE592427A0AEce92De3Edee1F18E0157C05861564,NaN,1.758530e+21,-6.248635e+18,0x4d246bE90C2f36730bb853aD41d0a189061192d3,1.769061e+22,NaN,NaN
2285,Swap,36,2,0xc1158d9d97e93a47483b144fdeafd4fd16916390d4a3...,0x157Dfa656Fdf0D18E1bA94075a53600D81cB3a97,0x3b329f60cf401398e32d5008bf588e2b676bf866632c...,12997793,4.723655e+27,-56398.0,None,NaN,NaN,0x911605012f87A3017322c81fCB4C90ADA7C09116,NaN,-4.877121e+20,1.736010e+18,0x911605012f87A3017322c81fCB4C90ADA7C09116,1.769061e+22,NaN,NaN
2286,Swap,111,90,0xe1876a9af128fe12c6e1bc0e07518cc721eaf31d61cc...,0x157Dfa656Fdf0D18E1bA94075a53600D81cB3a97,0x7eea24b22be33251175255623a878af1977ee79ccebf...,12998524,4.701978e+27,-56490.0,None,NaN,NaN,0xE592427A0AEce92De3Edee1F18E0157C05861564,NaN,1.372071e+21,-4.840290e+18,0x4A137FD5e7a256eF08A7De531A17D0BE0cc7B6b6,1.769061e+22,NaN,NaN


In [114]:
for i in range(len(df)):
    tdf = df.iloc[i]
    if tdf['event'] == 'Mint':
        break

In [115]:
tdf.dropna()

event                                                            Mint
logIndex                                                          209
transactionIndex                                                   84
transactionHash     0xa3a37265f2f428b110d89a0df297a47b537f77b9cceb...
address                    0x157Dfa656Fdf0D18E1bA94075a53600D81cB3a97
blockHash           0xa6f91c98be020ba125040ac0a9e4b7c4cc8bc2888d76...
blockNumber                                                  12376332
args.owner                 0xC36442b4a4522E871399CD717aBDD847Ab11FE88
args.tickLower                                               -49260.0
args.tickUpper                                               -49200.0
args.sender                0xC36442b4a4522E871399CD717aBDD847Ab11FE88
args.amount                                  1139244311526734823424.0
args.amount0                                    2000000000000000000.0
args.amount1                                     276977933973975808.0
Name: 1, dtype: obje

In [154]:
logIndex = tdf['logIndex']
transactionIndex = tdf['transactionIndex']
transactionHash = tdf['transactionHash']
blockNumber = tdf['blockNumber']
sender = tdf['args.sender']
tickLower = tdf['args.tickLower']
tickUpper = tdf['args.tickUpper']
amount = tdf['args.amount']
amount0 = tdf['args.amount0']
amount1 = tdf['args.amount1']

In [126]:
#data frame of current active positions
active_positions = pd.DataFrame()

In [158]:
mint_df = pd.DataFrame([[logIndex, blockNumber, transactionIndex, transactionHash, sender, amount, tickLower, tickUpper, amount0, amount1]], 
                       columns=['logIndex', 'blockNumber', 'transactionIndex', 'transactionHash', 'sender', 'amount', 'tickLower', 'tickUpper', 'amount0', 'amount1'])

In [161]:
add_active_df = pd.DataFrame([[logIndex, blockNumber, transactionIndex, transactionHash, sender, amount, tickLower, tickUpper, amount0, amount1]], 
                       columns=['logIndex', 'blockNumber', 'transactionIndex', 'transactionHash', 'sender', 'amount', 'tickLower', 'tickUpper', 'amount0', 'amount1'])

In [ ]:
#ticklower, tickupper, amount of liquidity, owner, token0 portfolio holdings, token1 portfolio holdings (start and last), fees0, fees1 total accrued, fees0, fee1 collected, last updated meta started meta  

In [ ]:
#factory contains to relant pool data, 
#pool init sets the price and tick
#

In [164]:
Camount0 = amount0
Camount1 = amount1

In [165]:
add_active_df = pd.DataFrame([[amount, tickLower, tickUpper, sender,
                               amount0, amount1, Camount0, Camount1,
                               0,0,0,0,
                               logIndex, blockNumber, transactionIndex, transactionHash,
                               logIndex, blockNumber, transactionIndex, transactionHash]],
                               columns = ['L', 'tickLower', 'tickUpper', 'owner', 'start_token0_holdings', 'start_token1_holdings',
                                            'last_token0_holdings', 'last_token1_holdings',
                                            'token0_fees_accrued', 'token1_fees_accrued',
                                            'token0_fees_collected', 'token1_fees_collected',
                                            'start_logIndex', 'start_blockNumber', 'start_transactionIndex', 'start_transactionHash', 
                                            'last_logIndex', 'last_blockNumber', 'last_transactionIndex', 'last_transactionHash'])

In [166]:
add_active_df

,L,tickLower,tickUpper,owner,start_token0_holdings,start_token1_holdings,last_token0_holdings,last_token1_holdings,token0_fees_accrued,token1_fees_accrued,token0_fees_collected,token1_fees_collected,start_logIndex,start_blockNumber,start_transactionIndex,start_transactionHash,last_logIndex,last_blockNumber,last_transactionIndex,last_transactionHash
0,1.139244e+21,-49260.0,-49200.0,0xC36442b4a4522E871399CD717aBDD847Ab11FE88,2.000000e+18,2.769779e+17,2.000000e+18,2.769779e+17,0,0,0,0,209,12376332,84,0xa3a37265f2f428b110d89a0df297a47b537f77b9cceb...,209,12376332,84,0xa3a37265f2f428b110d89a0df297a47b537f77b9cceb...


In [ ]:
columns=['L', 'tickLower', 'tickUpper', 'owner', 
         'start_token0_holdings', 'start_token1_holdings',
         'last_token0_holdings', 'last_token1_holdings',
         'token0_fees_accrued', 'token1_fees_accrued',
         'token0_fees_collected', 'token1_fees_collected',
         'start_logIndex', 'start_blockNumber', 'start_transactionIndex', 'start_transactionHash', 
         'last_logIndex', 'last_blockNumber', 'last_transactionIndex', 'last_transactionHash']